## Import libraries ##

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
from statsmodels.stats.proportion import proportion_confint, proportions_ztest
from sklearn.preprocessing import PowerTransformer, StandardScaler

## Custom modules
import sys
import os
os.environ['PATH'] = os.environ['PATH'] + ':/Library/TeX/texbin'
sys.path.append('/Users/riccardo/Documents/GitHub/ImmuneResponseCOVID19/')
from Modules import Parameters, DataPreprocessing as DP, CustomFunctions as CF
from Modules import CustomClasses as cc

## Colors

In [ ]:
## Color scheme
color_control = CF.lighten_color(Parameters.green, 0.8)
color_control2 = Parameters.lavander
color_immunecells = Parameters.orange
color_cytokines = Parameters.pink #red_purp
color_bloodtests1 = Parameters.lavander
color_bloodtests2 = Parameters.violet
color_scores = Parameters.light_red_purp #coral
color_demographics = Parameters.light_blue #pine_green
color_0 = Parameters.dark_blue
color_0_light = Parameters.light_blue
color_1 = Parameters.red_purp
color_1_light = Parameters.light_red_purp
color_background = CF.lighten_color(Parameters.light_grey, 0.6)

## Parameters

In [ ]:
## Multivariate sets
allinput_set = Parameters.allinput_set
immunecells_set = Parameters.immunecells_set
demographics_set = Parameters.demographics_set
cytokines_set = Parameters.cytokines_set 
biomarkers_set = Parameters.biomarkers_set
scores_set = Parameters.scores_set
comorbidities_set = Parameters.comorbidities_set

## Targets
target_train = Parameters.train_target
target_test = Parameters.test_target

## Age
lower_bound = Parameters.age_min
upper_bound = Parameters.age_max

## Delta onset
lower_bound_donset = Parameters.donset_min
upper_bound_donset = Parameters.donset_max

## Paths
path_results = Parameters.path_results
path_figures = Parameters.path_figures
foldername_univ = Parameters.foldername_univ

## Import and prepare datasets

In [ ]:
DataInpatients, DataOutpatients, DataHealthyCohort = DP.data_preprocessing(return_allfeatures=True, return_controlset=True)

## Filter data by target
mask = pd.notnull(DataInpatients[target_train].values) & pd.notnull(DataInpatients[target_test].values)
DataInpatients = DataInpatients.loc[mask,:]

print('\nInpatients dataset shape:', DataInpatients.shape)
print('Outpatients dataset shape:', DataOutpatients.shape)
print('Healthy cohort dataset shape:', DataHealthyCohort.shape)

## Flow cytometry sets

In [ ]:
flowcyt_set1 = ['NK/uL', 'B CD19/uL', 'T CD3/uL', 'T CD4/uL', '% T CD4 HLADR+', 'T CD8/uL', '% T CD8 HLADR+']
flowcyt_set2 = ['WBC/uL', 'Mono/uL', 'Mono DR IF', 'Mono DR %', 'Lymph/uL', 'RTE/uL', 'RTE % CD4']
flowcyt_set = flowcyt_set1 + flowcyt_set2

# Histograms

## Delta onset

In [ ]:
fontsize = 9
ylabelsize = 7
xlabelsize = 7 
tex = True
axes_lines_w = 0.5
lines_w = .5

In [ ]:
variable = 'delta_onset'

magnification = 0.55
ratio = 2.2/2.4
CF.SetPlotParams(magnification=magnification, ratio=ratio, 
                fontsize=fontsize, ylabelsize=ylabelsize, xlabelsize=xlabelsize, 
                tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w)

color = Parameters.light_grey
color_edge = Parameters.light_grey
bins = np.arange(-2, 35, 2)
v = DataInpatients[variable].dropna().values
y_hist, _, _ = plt.hist(v, bins=bins, color=color, alpha=0)
plt.hist(v, density=False, bins=bins, color=color, edgecolor='white', alpha=1., linewidth=0.9)
x_min, x_max = min(bins), max(bins)
plt.xlim([x_min, x_max])
plt.ylabel('patients')
plt.xlabel('$\mathrm{\Delta t_{ons}}$')
plt.xticks([0, 5, 10, 15, 20])
plt.xlim([-1, 21])
plt.grid(False)
plt.axvline(np.quantile(v, 0.5), lw=0.7, color='black', ls='--', dashes=(4, 4))
saving_str = path_figures + 'cases_vs_donset.pdf'
plt.savefig(saving_str, bbox_inches='tight')# dpi=1000,
plt.show()

## WHO

In [ ]:
fontsize = 9
ylabelsize = 7
xlabelsize = 7 
tex = True
axes_lines_w = 0.5
lines_w = .5

In [ ]:
variable = 'WHO'

magnification = 0.43
ratio = 2.4/2.2
CF.SetPlotParams(magnification=magnification, ratio=ratio, 
                fontsize=fontsize, ylabelsize=ylabelsize, xlabelsize=xlabelsize, 
                tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w)

color = Parameters.light_grey
color_edge = Parameters.light_grey
bins = np.arange(-0.5, 6.5, 1)
v = DataInpatients[variable].dropna().values
v[v==0] = 1
v[v==5] = 4
y_hist, _, _ = plt.hist(v, bins=bins, color=color, alpha=0)
plt.hist(v, density=False, bins=bins, color=color, edgecolor='white', alpha=0.9, linewidth=1.5)
x_min, x_max = min(bins), max(bins)
plt.xlim([x_min, x_max])
plt.ylabel('n. patients')
plt.xlabel('WHO')
plt.xticks([1, 2, 3, 4])
plt.xlim([0., 5])
plt.yticks([0, 100, 200, 300])
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.grid(False)
saving_str = path_figures + 'cases_vs_WHO.pdf'
plt.show()

## Age

In [ ]:
variable = 'age'

Data = DataInpatients.copy()
Data_c = DataOutpatients.copy()

magnification = 0.55
ratio = 2.2/2.4
CF.SetPlotParams(magnification=magnification, ratio=ratio, 
                fontsize=fontsize, ylabelsize=ylabelsize, xlabelsize=xlabelsize, 
                tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w)

fig, ax = plt.subplots()

v = Data[variable].dropna().values
x_1_min, x_1_max = 10, 110
bins = np.linspace(x_1_min, x_1_max, 13)
color = Parameters.light_grey
ax.hist(v, bins,
        alpha=1., 
        color=color, 
        density=False,
        edgecolor='white',
        label='COVID-19')
plt.axvline(np.quantile(v, 0.5), lw=0.7, color='black', ls='--', dashes=(4, 4))
ax.set_xlabel('age')
ax.set_xticks([20, 40, 60, 80, 100])
ax.set_yticks([0, 100, 200])
ax.set_xlim([15, 105])
ax.set_ylabel('n. patients')
saving_str = path_figures + 'inpatients_vs_age' + '.pdf'
plt.savefig(saving_str, bbox_inches='tight')
plt.show()

In [ ]:
variable = 'age'

Data = DataInpatients.copy()
Data_c = DataOutpatients.copy()
Data_h = DataHealthyCohort.copy()

magnification = 0.55
ratio = 2.2/2.4
CF.SetPlotParams(magnification=magnification, ratio=ratio, 
                fontsize=fontsize, ylabelsize=ylabelsize, xlabelsize=xlabelsize, 
                tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w)

fig, ax = plt.subplots()

v1 = Data[variable]
v2 = Data_c[variable]
x_1_min, x_1_max = 10, 110
bins = np.linspace(x_1_min, x_1_max, 13)
color = color_immunecells
ax.hist(v1, bins,
        alpha=0.2, 
        histtype='stepfilled',
        color=color, 
        density=False,
        edgecolor='white',
        label='COVID-19')
ax.hist(v1, bins,
        alpha=1., 
        histtype='step',
        edgecolor=color,
        density=False,
        facecolor='None')
color = color_control
ax.hist(v2, bins,
        alpha=.2, 
        histtype='stepfilled',
        color=color,
        density=False,
        edgecolor=color, 
        label='control')
ax.hist(v2, bins,
        alpha=1., 
        histtype='step',
        edgecolor=color,
        density=False,
        facecolor='None')
ax.set_xlabel('age')
ax.set_xticks([20, 40, 60, 80, 100])
ax.set_xlim([5, 105])
ax.set_ylabel('n. patients')
saving_str = path_figures + 'cases_vs_age' + '.pdf'
plt.savefig(saving_str, bbox_inches='tight')# dpi=1000,
plt.show()

## Age - outpatients and healthy cohort

In [ ]:
variable = 'age'

sc = StandardScaler()
pt = PowerTransformer()

magnification = 0.55
ratio = 2.2/2.5
CF.SetPlotParams(magnification=magnification, ratio=ratio, 
                fontsize=fontsize, ylabelsize=ylabelsize, xlabelsize=xlabelsize, 
                tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w)

fig, ax = plt.subplots()

v1 = Data_h[variable].dropna().to_numpy()
v2 = Data_c[variable].dropna().to_numpy()
mw_pval = st.mannwhitneyu(v1, v2)[1]
v1 = pt.fit_transform(sc.fit_transform(v1.reshape(-1,1))).reshape(-1,)
v2 = pt.transform(sc.transform(v2.reshape(-1,1))).reshape(-1,)
bins = np.linspace(-4.5, 4.5, 18)
color = color_control2 
ax.hist(v1, bins,
        alpha=.1,
        histtype='stepfilled',
        color=color,
        density=True,
        edgecolor=color, 
        label='control')
ax.hist(v1, bins,
        alpha=1.,
        histtype='step',
        edgecolor=color,
        density=True,
        facecolor='None')
color = color_control
ax.hist(v2, bins,
        alpha=0.15, 
        histtype='stepfilled',
        color=color, 
        density=True,
        edgecolor='white',
        label='COVID-19')
ax.hist(v2, bins,
        alpha=1.,
        histtype='step',
        edgecolor=color,
        density=True,
        facecolor='None')
ax.set_xlabel('%s' % CF.change_names([variable], units=False)[0])
ax.set_ylabel('density')
ax.set_xticks([-3, 0, 3])
ax.set_xlim([-5., 5.])
ax.set_yticks([0, 0.4, 0.8])
ax.set_ylim([0, 0.8])
p_val_str = 'p=%.3f' % mw_pval
plt.text(.99, .99, p_val_str, ha='right', va='top', transform=ax.transAxes)

saving_str = path_figures + 'cases_vs_age_outp' + '.pdf'
plt.savefig(saving_str, bbox_inches='tight')# dpi=1000,
plt.show()

In [ ]:
magnification = 0.55
ratio = 2.2/2.5
CF.SetPlotParams(magnification=magnification, ratio=ratio, 
                fontsize=fontsize, ylabelsize=ylabelsize, xlabelsize=xlabelsize, 
                tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w)

fig, ax = plt.subplots()

v1 = Data_h[variable].dropna().to_numpy()
v2 = Data[variable].dropna().to_numpy()
mw_pval = st.mannwhitneyu(v1, v2)[1]
v1 = pt.fit_transform(sc.fit_transform(v1.reshape(-1,1))).reshape(-1,)
v2 = pt.transform(sc.transform(v2.reshape(-1,1))).reshape(-1,)
bins = np.linspace(-4.5, 4.5, 18)
color = color_control2 
ax.hist(v1, bins,
        alpha=.1,
        histtype='stepfilled',
        color=color,
        density=True,
        edgecolor=color, 
        label='control')
ax.hist(v1, bins,
        alpha=1.,
        histtype='step',
        edgecolor=color,
        density=True,
        facecolor='None')
color = color_immunecells
ax.hist(v2, bins,
        alpha=0.15, 
        histtype='stepfilled',
        color=color, 
        density=True,
        edgecolor='white',
        label='COVID-19')
ax.hist(v2, bins,
        alpha=1.,
        histtype='step',
        edgecolor=color,
        density=True,
        facecolor='None')

ax.set_xlabel('%s' % CF.change_names([variable], units=False)[0])
ax.set_ylabel('density')
ax.set_xticks([-3, 0, 3])
ax.set_xlim([-5., 5.])
ax.set_yticks([0, 0.4, 0.8])
ax.set_ylim([0, 0.8])
p_val_str = 'p=%.3f' % mw_pval
plt.text(.99, .99, p_val_str, ha='right', va='top', transform=ax.transAxes)
saving_str = path_figures + 'cases_vs_age_inphp' + '.pdf'
plt.savefig(saving_str, bbox_inches='tight')# dpi=1000,
plt.show()

## Sex - outpatients and healthy cohort

In [ ]:
variable = 'sex'

CF.SetPlotParams(magnification=magnification*0.86, ratio=ratio, 
                fontsize=fontsize, ylabelsize=ylabelsize, xlabelsize=xlabelsize, 
                tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w)

v1 = Data_h[variable].dropna().to_numpy()
v2 = Data_c[variable].dropna().to_numpy()
count = np.array([np.sum(v1==1), np.sum(v2==1)])
nobs = np.array([np.sum(pd.notnull(v1)), np.sum(pd.notnull(v2==1))])
z_pval = proportions_ztest(count=count, nobs=nobs)[1]
frac_male_1 = sum(v1==0)/sum(pd.notnull(v1))
ci1_low, ci1_up = proportion_confint(sum(v1==0), sum(pd.notnull(v1)), alpha=0.05, method='binom_test')
frac_male_2 = sum(v2==0)/sum(pd.notnull(v2))
ci2_low, ci2_up = proportion_confint(sum(v2==0), sum(pd.notnull(v2)), alpha=0.05, method='binom_test')
perc_male_1 = round(frac_male_1*100)
yerr_perc_male_1 = np.array([(frac_male_1 - ci1_low)*100, (ci1_up - frac_male_1)*100]).reshape(-1,1)
perc_male_2 = round(frac_male_2*100)
yerr_perc_male_2 = np.array([(frac_male_2 - ci2_low)*100, (ci2_up - frac_male_2)*100]).reshape(-1,1)

fig, ax = plt.subplots()

widths = 0.3
lines_w = 0.5
capsize = 0
ax.bar(0, perc_male_1, yerr=yerr_perc_male_1, color=color_control2, alpha=0.65, width=widths, lw=lines_w, capsize=capsize)
ax.bar(0.7, perc_male_2, yerr=yerr_perc_male_2, color=color_control, alpha=0.65, width=widths, lw=lines_w, capsize=capsize)
p_val_str = 'p=%.3f' % z_pval
plt.text(.99, .99, p_val_str, ha='right', va='top', transform=ax.transAxes)
plt.yticks([0, 50, 100])
plt.ylim([0, 100])
plt.xticks([0, 0.7], ['h', 'o'], )
plt.xlim([-0.5, 1.2])
plt.ylabel('\% male')
saving_str = path_figures + 'sex_outp' + '.pdf'
plt.savefig(saving_str, bbox_inches='tight')# dpi=1000,
plt.show()

In [ ]:
variable = 'sex'

CF.SetPlotParams(magnification=magnification*0.86, ratio=ratio, 
                fontsize=fontsize, ylabelsize=ylabelsize, xlabelsize=xlabelsize, 
                tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w)

v1 = Data_h[variable].dropna().to_numpy()
v2 = Data[variable].dropna().to_numpy()
count = np.array([np.sum(v1==1), np.sum(v2==1)])
nobs = np.array([np.sum(pd.notnull(v1)), np.sum(pd.notnull(v2==1))])
z_pval = proportions_ztest(count=count, nobs=nobs)[1]

frac_male_1 = sum(v1==0)/sum(pd.notnull(v1))
ci1_low, ci1_up = proportion_confint(sum(v1==0), sum(pd.notnull(v1)), alpha=0.05, method='binom_test')
frac_male_2 = sum(v2==0)/sum(pd.notnull(v2))
ci2_low, ci2_up = proportion_confint(sum(v2==0), sum(pd.notnull(v2)), alpha=0.05, method='binom_test')
perc_male_1 = round(frac_male_1*100)
yerr_perc_male_1 = np.array([(frac_male_1 - ci1_low)*100, (ci1_up - frac_male_1)*100]).reshape(-1,1)
perc_male_2 = round(frac_male_2*100)
yerr_perc_male_2 = np.array([(frac_male_2 - ci2_low)*100, (ci2_up - frac_male_2)*100]).reshape(-1,1)

fig, ax = plt.subplots()

widths = 0.3
lines_w = 0.5
capsize = 0
ax.bar(0, perc_male_1, yerr=yerr_perc_male_1, color=color_control2, alpha=0.65, width=widths, lw=lines_w, capsize=capsize)
ax.bar(0.7, perc_male_2, yerr=yerr_perc_male_2, color=color_immunecells, alpha=0.65, width=widths, lw=lines_w, capsize=capsize)
p_val_str = 'p=%.3f' % z_pval
plt.text(.99, .99, p_val_str, ha='right', va='top', transform=ax.transAxes)
plt.yticks([0, 50, 100])
plt.ylim([0, 100])
plt.xticks([0, 0.7], ['h', 'in'], )
plt.xlim([-0.5, 1.2])
plt.ylabel('\% male')
saving_str = path_figures + 'sex_inphp' + '.pdf'
plt.savefig(saving_str, bbox_inches='tight')# dpi=1000,
plt.show()

## Flow cytometry set - outpatients and healthy cohort

In [ ]:
sc = StandardScaler()
pt = PowerTransformer()

for idx, variable in enumerate(immunecells_set):
    print(variable)
    
    magnification = 0.55
    ratio = 2.2/2.5
    CF.SetPlotParams(magnification=magnification, ratio=ratio, 
                     fontsize=fontsize, ylabelsize=ylabelsize, xlabelsize=xlabelsize, 
                     tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w)

    fig, ax = plt.subplots()
    
    v1 = Data_h[variable].dropna().to_numpy()
    v2 = Data_c[variable].dropna().to_numpy()
    mw_pval = st.mannwhitneyu(v1, v2)[1]
    v1 = pt.fit_transform(sc.fit_transform(v1.reshape(-1,1))).reshape(-1,)
    v2 = pt.transform(sc.transform(v2.reshape(-1,1))).reshape(-1,)
    bins = np.linspace(-5, 5, 18)
    color = color_control2 
    ax.hist(v1, bins,
            alpha=.1,
            histtype='stepfilled',
            color=color,
            density=True,
            edgecolor=color, 
            label='control')
    ax.hist(v1, bins,
            alpha=1.,
            histtype='step',
            edgecolor=color,
            density=True,
            facecolor='None')
    color = color_control
    ax.hist(v2, bins,
            alpha=0.15, 
            histtype='stepfilled',
            color=color, 
            density=True,
            edgecolor='white',
            label='COVID-19')
    ax.hist(v2, bins,
            alpha=1.,
            histtype='step',
            edgecolor=color,
            density=True,
            facecolor='None')
    ax.set_xlabel('%s' % CF.change_names([variable], units=False)[0])
    ax.set_ylabel('density')
    ax.set_xticks([-3, 0, 3])
    ax.set_xlim([-5., 5.])
    ax.set_yticks([0, 0.4, 0.8])
    ax.set_ylim([0, 0.8])
    p_val_str = 'p=%.3f' % mw_pval
    plt.text(.99, .99, p_val_str, ha='right', va='top', transform=ax.transAxes)
    saving_str = path_figures + 'cases_vs_FC#%d_outp' % idx + '.pdf'
    plt.savefig(saving_str, bbox_inches='tight')# dpi=1000,
    plt.show()

In [ ]:
sc = StandardScaler()
pt = PowerTransformer()

for idx, variable in enumerate(immunecells_set):
    print(variable)
    
    magnification = 0.55
    ratio = 2.2/2.5
    CF.SetPlotParams(magnification=magnification, ratio=ratio, 
                     fontsize=fontsize, ylabelsize=ylabelsize, xlabelsize=xlabelsize, 
                     tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w)

    fig, ax = plt.subplots()
    
    v1 = Data_h[variable].dropna().to_numpy()
    v2 = Data[variable].dropna().to_numpy()
    mw_pval = st.mannwhitneyu(v1, v2)[1]
    v1 = pt.fit_transform(sc.fit_transform(v1.reshape(-1,1))).reshape(-1,)
    v2 = pt.transform(sc.transform(v2.reshape(-1,1))).reshape(-1,)
    bins = np.linspace(-5, 5, 18)
    color = color_control2 
    ax.hist(v1, bins,
            alpha=.1,
            histtype='stepfilled',
            color=color,
            density=True,
            edgecolor=color, 
            label='control')
    ax.hist(v1, bins,
            alpha=1.,
            histtype='step',
            edgecolor=color,
            density=True,
            facecolor='None')

    color = color_immunecells
    ax.hist(v2, bins,
            alpha=0.15, 
            histtype='stepfilled',
            color=color, 
            density=True,
            edgecolor='white',
            label='COVID-19')
    ax.hist(v2, bins,
            alpha=1.,
            histtype='step',
            edgecolor=color,
            density=True,
            facecolor='None')

    ax.set_xlabel('%s' % CF.change_names([variable], units=False)[0])
    ax.set_ylabel('density')
    ax.set_xticks([-3, 0, 3])
    ax.set_xlim([-5., 5.])
    ax.set_yticks([0, 0.4, 0.8])
    ax.set_ylim([0, 0.8])
    p_val_str = 'p=%.3f' % mw_pval
    plt.text(.99, .99, p_val_str, ha='right', va='top', transform=ax.transAxes)

    saving_str = path_figures + 'cases_vs_FC#%d_inphp' % idx + '.pdf'
    #print(saving_str)
    plt.savefig(saving_str, bbox_inches='tight')# dpi=1000,
    plt.show()

# Low-risk variables: 1\% - 99\% quantiles

In [ ]:
features_list = ['Mono DR IF', 'RTE/uL', 
                 'IL10', 'IL2R', 'IL6', 'IL8', 'IP10', 
                 'proADM', 'CRP']

for name in features_list:
    v = DataInpatients[name].dropna()
    q1, q99 = np.quantile(v, [0.01, 0.99])
    min_val, max_val = min(v), max(v)
    print('%s (%.2f) %.2f-%.2f (%.2f)' %(name, min_val, q1, q99, max_val))

# Tables

### Summary table

In [ ]:
## flowcyt_set, demographics, prog_scores_2, low_VIF_cy_sel_0, blood_tests

column_set_dict = {'FC': immunecells_set,
                   'CK': cytokines_set, 
                   'BM': biomarkers_set, 
                   'C': immunecells_set,}

table_dict1 = {'N': [], 'sex (female)': [], 'age [Q2(Q1-Q3)]': [], 'd. onset [Q2(Q1-Q3)]': [], 'CCI<2': [], 'NANs': []}
table_dict2 = {'SOFA>1': [], 'NEWS>4': [], 'qCSI>6': [], '4C>8': [], 'WHO>2': [], 'OTI+death': []}

for set_key in column_set_dict.keys():
    
    table_dict = {}
    column_set = column_set_dict[set_key]
    columns = demographics_set + column_set + scores_set + ['OTI+death', 'WHO']
    if set_key=='C':
        Data = DataOutpatients[columns].copy()
    else:
        Data = DataInpatients[columns].copy()
    mask = 100*np.sum(Data[column_set].isnull(), axis=1)/len(column_set)<=50
    Data = Data.loc[mask,:]
    # first row
    table_dict1["N"].append(Data.shape[0])
    table_dict1["sex (female)"].append("%.1f%%" % (np.nansum(Data["sex"])/np.sum(Data["sex"].notnull())*100))
    table_dict1["age [Q2(Q1-Q3)]"].append("%.f(%.f-%.f)" % (np.nanquantile(Data["age"],0.5), np.nanquantile(Data["age"],0.25), np.nanquantile(Data["age"],0.75)))
    table_dict1["d. onset [Q2(Q1-Q3)]"].append("%.f(%.f-%.f)" % (np.nanquantile(Data["delta_onset"],0.5), np.nanquantile(Data["delta_onset"],0.25), np.nanquantile(Data["delta_onset"],0.75)))
    table_dict1["CCI<2"].append("%.1f%%" % (np.nansum(Data["CCI (charlson comorbidity index)"]<2)/np.sum(Data["CCI (charlson comorbidity index)"].notnull())*100))
    table_dict1["NANs"].append("%.1f%%" % (Data[column_set].isnull().sum().sum()/Data[column_set].size*100))
    # second row
    table_dict2["SOFA>1"].append("%.1f%%" % (np.nansum(Data["SOFA"]>1)/np.sum(Data["SOFA"].notnull())*100))
    table_dict2["NEWS>4"].append("%.1f%%" % (np.nansum(Data["NEWS"]>4)/np.sum(Data["NEWS"].notnull())*100))
    table_dict2["qCSI>6"].append("%.1f%%" % (np.nansum(Data["qCSI"]>6)/np.sum(Data["qCSI"].notnull())*100))
    table_dict2["4C>8"].append("%.1f%%" % (np.nansum(Data["4C"]>8)/np.sum(Data["4C"].notnull())*100))
    table_dict2["WHO>2"].append("%.1f%%" % (np.nansum(Data["WHO"]>2)/np.sum(Data["WHO"].notnull())*100))
    table_dict2["OTI+death"].append("%.1f%%" % (np.nansum(Data["OTI+death"]>0)/np.sum(Data["OTI+death"].notnull())*100))

table_df1 = pd.DataFrame(table_dict1, index=['FC', 'CK', 'BM', 'C'])
table_df2 = pd.DataFrame(table_dict2, index=['FC', 'CK', 'BM', 'C'])

In [ ]:
table_df1

In [ ]:
table_df2

### Comprehensive table

In [ ]:
Data_l = DataInpatients.copy()

Data_l['control'] = 0
Data_c = DataOutpatients.copy()
Data_c['control'] = 1
Data_complete = pd.concat([Data_l, Data_c], ignore_index=True)
Data_complete.reset_index(drop=True, inplace=True)


## define masks
age_thr = 70
donset_th = 10
group_masks = {}
# control
control_mask = Data_complete['control']==1
mask_control_age_l = (control_mask) & (Data_complete['age']<age_thr)
mask_control_age_h = (control_mask) & (Data_complete['age']>=age_thr)
group_masks['control'] = control_mask
group_masks['control_age_l'] = mask_control_age_l
group_masks['control_age_h'] = mask_control_age_h
# covid
covid_mask = Data_complete['control']==0
mask_covid_age_l = (covid_mask) & (Data_complete['age']<age_thr)
mask_covid_age_h = (covid_mask) & (Data_complete['age']>=age_thr)
mask_covid_donset_l = (covid_mask) & (Data_complete['delta_onset']<=donset_th)
mask_covid_donset_h = (covid_mask) & (Data_complete['delta_onset']>donset_th)
group_masks['covid'] = covid_mask
group_masks['covid_age_l'] = mask_covid_age_l
group_masks['covid_age_h'] = mask_covid_age_h
group_masks['covid_donset_l'] = mask_covid_donset_l
group_masks['covid_donset_h'] = mask_covid_donset_h
# covid who
mask_covid_who_02 = (covid_mask) & ((0<=Data_complete['WHO']) & (Data_complete['WHO']<=2))
mask_covid_who_35 = (covid_mask) & (Data_complete['WHO']>=3)
group_masks['covid_who_02'] = mask_covid_who_02
group_masks['covid_who_45'] = mask_covid_who_35


## Demographics
description_dict = {}
name = 'Number'
values = []
for group_mask in group_masks.keys():
    mask = (group_masks[group_mask]) #& pd.notnull(Data_complete['ID'])
    n = sum(mask)
    value_str = str(n)
    values.append(value_str)
description_dict[name] = values
numbers = np.array(list(map(lambda x: int(x), values)))

name = 'Age'
column = 'age'
values = []
for group_mask in group_masks.keys():
    mask = (group_masks[group_mask]) & pd.notnull(Data_complete[column])
    v = Data_complete.loc[mask, column].values
    q1, median, q3 = Data_complete.loc[mask, column].quantile([0.25,0.5,0.75])
    value_str = '%d (%d-%d)' % (round(median), round(q1), round(q3))
    values.append(value_str)
description_dict[name] = values

name = 'Male'
column = 'sex'
values = []
for i, group_mask in enumerate(group_masks.keys()):
    mask = (group_masks[group_mask]) & pd.notnull(Data_complete[column])
    v = Data_complete.loc[mask, column].values
    value = np.sum(v==0)
    number = len(v)
    perc = value/number
    value_str = '%d/%d (%.0f%%)' % (value, number, 100*perc)
    values.append(value_str)
description_dict[name] = values

name = '$\mathrm{\Delta t_{onset}}$[Q2(Q1-Q3)]'
column = 'delta_onset'
values = []
for group_mask in group_masks.keys():
    mask = (group_masks[group_mask]) & pd.notnull(Data_complete[column])
    v = Data_complete.loc[mask, column].values
    if len(v)<20 or value==0:
        value_str = '-'
    else:
        q1, median, q3 = Data_complete.loc[mask, column].quantile([0.25,0.5,0.75])
        value_str = '%d (%d-%d)' % (round(median), round(q1), round(q3))
    values.append(value_str)
description_dict[name] = values


## Comorbidities
name = 'Obesity'
column = 'obesity'
values = []
max_perc = 0
for i, group_mask in enumerate(group_masks.keys()):
    mask = (group_masks[group_mask]) & pd.notnull(Data_complete[column])
    v = Data_complete.loc[mask, column].values
    value = np.sum(v==1)
    if len(v)<20 or value==0:
        value_str = '-'
    else:
        number = len(v)
        perc = value/number
        value_str = '%d/%d (%.0f%%)' % (value, number, 100*perc)
        if perc>max_perc:
            max_perc = perc
    values.append(value_str)
description_dict[name] = values

name = 'Hypertension'
column = 'hypertension'
values = []
max_perc = 0
for i, group_mask in enumerate(group_masks.keys()):
    mask = (group_masks[group_mask]) & pd.notnull(Data_complete[column])
    v = Data_complete.loc[mask, column].values
    value = np.sum(v==1)
    if len(v)<20 or value==0:
        value_str = '-'
    else:
        number = len(v)
        perc = value/number
        value_str = '%d/%d (%.0f%%)' % (value, number, 100*perc)
        if perc>max_perc:
            max_perc = perc
    values.append(value_str)
description_dict[name] = values

name = 'CVDs'
column = 'CVDs'
values = []
max_perc = 0
for i, group_mask in enumerate(group_masks.keys()):
    mask = (group_masks[group_mask]) & pd.notnull(Data_complete[column])
    v = Data_complete.loc[mask, column].values
    value = np.sum(v==1)
    if len(v)<20 or value==0:
        value_str = '-'
    else:
        number = len(v)
        perc = value/number
        value_str = '%d/%d (%.0f%%)' % (value, number, 100*perc)
        if perc>max_perc:
            max_perc = perc
    values.append(value_str)
description_dict[name] = values

name = 'Dyslipidemia'
column = 'dyslipidemia'
values = []
max_perc = 0
for i, group_mask in enumerate(group_masks.keys()):
    mask = (group_masks[group_mask]) & pd.notnull(Data_complete[column])
    v = Data_complete.loc[mask, column].values
    value = np.sum(v==1)
    if len(v)<20 or value==0:
        value_str = '-'
    else:
        number = len(v)
        perc = value/number
        value_str = '%d/%d (%.0f%%)' % (value, number, 100*perc)
        if perc>max_perc:
            max_perc = perc
    values.append(value_str)
description_dict[name] = values

name = 'Diabetes'
column = 'diabetes'
values = []
max_perc = 0
for i, group_mask in enumerate(group_masks.keys()):
    mask = (group_masks[group_mask]) & pd.notnull(Data_complete[column])
    v = Data_complete.loc[mask, column].values
    value = np.sum(v==1)
    if len(v)<20 or value==0:
        value_str = '-'
    else:
        number = len(v)
        perc = value/number
        value_str = '%d/%d (%.0f%%)' % (value, number, 100*perc)
        if perc>max_perc:
            max_perc = perc
    values.append(value_str)
description_dict[name] = values

name = 'CKI'
column = 'CKI'
values = []
max_perc = 0
for i, group_mask in enumerate(group_masks.keys()):
    mask = (group_masks[group_mask]) & pd.notnull(Data_complete[column])
    v = Data_complete.loc[mask, column].values
    value = np.sum(v==1)
    if len(v)<20 or value==0:
        value_str = '-'
    else:
        number = len(v)
        perc = value/number
        value_str = '%d/%d (%.0f%%)' % (value, number, 100*perc)
        if perc>max_perc:
            max_perc = perc
    values.append(value_str)
description_dict[name] = values

name = 'Tumor'
column = 'tumor'
values = []
max_perc = 0
for i, group_mask in enumerate(group_masks.keys()):
    mask = (group_masks[group_mask]) & pd.notnull(Data_complete[column])
    v = Data_complete.loc[mask, column].values
    value = np.sum(v==1)
    if len(v)<20 or value==0:
        value_str = '-'
    else:
        number = len(v)
        perc = value/number
        value_str = '%d/%d (%.0f%%)' % (value, number, 100*perc)
        if perc>max_perc:
            max_perc = perc
    values.append(value_str)
description_dict[name] = values

name = 'COPD'
column = 'COPD'
values = []
max_perc = 0
for i, group_mask in enumerate(group_masks.keys()):
    mask = (group_masks[group_mask]) & pd.notnull(Data_complete[column])
    v = Data_complete.loc[mask, column].values
    value = np.sum(v==1)
    if len(v)<20 or value==0:
        value_str = '-'
    else:
        number = len(v)
        perc = value/number
        value_str = '%d/%d (%.0f%%)' % (value, number, 100*perc)
        if perc>max_perc:
            max_perc = perc
    values.append(value_str)
description_dict[name] = values

name = 'Autoimmunity'
column = 'autoimmunity'
values = []
max_perc = 0
for i, group_mask in enumerate(group_masks.keys()):
    mask = (group_masks[group_mask]) & pd.notnull(Data_complete[column])
    v = Data_complete.loc[mask, column].values
    value = np.sum(v==1)
    if len(v)<20 or value==0:
        value_str = '-'
    else:
        number = len(v)
        perc = value/number
        value_str = '%d/%d (%.0f%%)' % (value, number, 100*perc)
        if perc>max_perc:
            max_perc = perc
    values.append(value_str)
description_dict[name] = values

name = 'Hepatopathy'
column = 'hepatopathy'
values = []
max_perc = 0
for i, group_mask in enumerate(group_masks.keys()):
    mask = (group_masks[group_mask]) & pd.notnull(Data_complete[column])
    v = Data_complete.loc[mask, column].values
    value = np.sum(v==1)
    if len(v)<20 or value==0:
        value_str = '-'
    else:
        number = len(v)
        perc = value/number
        value_str = '%d/%d (%.0f%%)' % (value, number, 100*perc)
        if perc>max_perc:
            max_perc = perc
    values.append(value_str)
description_dict[name] = values

name = 'Oncohematology'
column = 'oncohematology'
values = []
max_perc = 0
for i, group_mask in enumerate(group_masks.keys()):
    mask = (group_masks[group_mask]) & pd.notnull(Data_complete[column])
    v = Data_complete.loc[mask, column].values
    value = np.sum(v==1)
    if len(v)<20 or value==0:
        value_str = '-'
    else:
        number = len(v)
        perc = value/number
        value_str = '%d/%d (%.0f%%)' % (value, number, 100*perc)
        if perc>max_perc:
            max_perc = perc
    values.append(value_str)
description_dict[name] = values

name = 'Immunosuppressed'
column = 'immunosuppressed'
values = []
max_perc = 0
for i, group_mask in enumerate(group_masks.keys()):
    mask = (group_masks[group_mask]) & pd.notnull(Data_complete[column])
    v = Data_complete.loc[mask, column].values
    value = np.sum(v==1)
    if len(v)<20 or value==0:
        value_str = '-'
    else:
        number = len(v)
        perc = value/number
        value_str = '%d/%d (%.0f%%)' % (value, number, 100*perc)
        if perc>max_perc:
            max_perc = perc
    values.append(value_str)
description_dict[name] = values


## CCI
name = 'CCI'
column = 'CCI (charlson comorbidity index)'
values = []
for i, group_mask in enumerate(group_masks.keys()):
    mask = (group_masks[group_mask]) & pd.notnull(Data_complete[column])
    v = Data_complete.loc[mask, column].values
    if len(v)<20:
        value_str = '-'
    else:
        q1, median, q3 = pd.Series(v).quantile([0.25,0.5,0.75])
        value_str = '%d (%d-%d)' % (median, q1, q3)
    values.append(value_str)
description_dict[name] = values


## Flow cytometry variables
flowcyt_sel_columns = ['WBC/uL', 
                       'Mono/uL', 'Mono DR IF', 'Mono DR %',
                       'Lymph/uL',
                       'T CD3/uL', 'T CD3 %', 'T CD3 HLADR/uL', 
                       'T CD4/uL', 'T CD4 %', '% T CD4 HLADR+', 'RTE % CD4', 'RTE/uL', 
                       'T CD8/uL', 'T CD8 %', '% T CD8 HLADR+', 
                       'B CD19/uL', 'B CD19 %', 
                       'NK/uL', 'NK %']
flowcyt_names = CF.change_names(flowcyt_sel_columns, units=True)
Data = Data_complete.loc[:, flowcyt_sel_columns].copy()

for i, col in enumerate(flowcyt_sel_columns):
    name = flowcyt_names[i]
    column = col
    values = []
    # find order of magnitude
    if '%' not in col:
        mask = (group_masks['covid']) & pd.notnull(Data[column])
        v = Data.loc[mask, column].values.reshape(-1,)
        # 
        median_ref = pd.Series(v).quantile(0.5)
        div = 1.
        while median_ref/div>1:
            div = div*10.
        div = div/10.
        exponent = int(np.log10(div))
        name_split = name.split(' ')
        name = name_split[0] + ' ($10^{%d}$ '%exponent + name_split[1] + ')'
    
    elif ' %' in col:
        name = name.replace(' %', ' (% Linfo)')
        
    for group_mask in group_masks.keys():
        mask = (group_masks[group_mask]) & pd.notnull(Data[column])
        v = Data.loc[mask, column].values
        #
        q1, median, q3 = pd.Series(v).quantile([0.25,0.5,0.75])
        if div>1 and '%' not in col:
            q1, median, q3 = q1/div, median/div, q3/div
        value_str = '%.1f (%.1f-%.1f)' % (median, q1, q3)
        values.append(value_str)
    description_dict[name] = values
    

## Cytokines
Data = Data_complete.loc[:, cytokines_set].copy()
cytokines_names = CF.change_names(cytokines_set, units=True)
for i, col in enumerate(cytokines_set):
    name = cytokines_names[i]
    column = col
    values = []
    # find order of magnitude
    mask = (group_masks['covid']) & pd.notnull(Data[column])
    v = Data.loc[mask, column].values.reshape(-1,)
    median_ref = pd.Series(v).quantile(0.5)
    div = 1.
    while median_ref/div>1:
        div = div*10.
    div = div/10.
    exponent = int(np.log10(div))
    name_split = name.split(' ')
    name = name_split[0] + ' ($10^{%d}$ '%exponent + name_split[1] + ')'
        
    for group_mask in group_masks.keys():
        mask = (group_masks[group_mask]) & pd.notnull(Data[column])
        v = Data.loc[mask, column].values.reshape(-1,)
        if len(v)<20:
            value_str = '-'
        else:
            q1, median, q3 = pd.Series(v).quantile([0.25,0.5,0.75])
            if div>1 and '%' not in col:
                q1, median, q3 = q1/div, median/div, q3/div
            value_str = '%.1f (%.1f-%.1f)' % (median, q1, q3)
        values.append(value_str)
    description_dict[name] = values


## Serological biomarkers
Data = Data_complete.loc[:, biomarkers_set].copy()
blood_tests_names = CF.change_names(biomarkers_set, units=True)
for i, col in enumerate(biomarkers_set):
    name = blood_tests_names[i]
    column = col
    values = []
    # find order of magnitude
    mask = (group_masks['covid']) & pd.notnull(Data[column])
    v = Data.loc[mask, column].values.reshape(-1,)
    # 
    median_ref = pd.Series(v).quantile(0.5)
    div = 1.
    while median_ref/div>1:
        div = div*10.
    div = div/10.
    exponent = int(np.log10(div))
    name_split = name.split(' ')
    name = name_split[0] + ' ($10^{%d}$ '%exponent + name_split[1] + ')'
    
    for group_mask in group_masks.keys():
        mask = (group_masks[group_mask]) & pd.notnull(Data[column])
        v = Data.loc[mask, column].values.reshape(-1,)
        if len(v)<20:
            value_str = '-'
        else:
            q1, median, q3 = pd.Series(v).quantile([0.25,0.5,0.75])
            if div>1 and '%' not in col:
                q1, median, q3 = q1/div, median/div, q3/div
            value_str = '%.1f (%.1f-%.1f)' % (median, q1, q3)
        values.append(value_str)
    description_dict[name] = values


## Outcome
name = 'Infectious complications'
column = 'infectious_complications'
values = []
for i, group_mask in enumerate(group_masks.keys()):
    mask = (group_masks[group_mask]) & pd.notnull(Data_complete[column])
    v = Data_complete.loc[mask, column].values
    value = np.sum(v==1)
    if len(v)<20 or value==0:
        value_str = '-'
    else:
        number = len(v)
        perc = value/number
        value_str = '%d/%d (%.0f%%)' % (value, number, 100*perc)
    values.append(value_str)
description_dict[name] = values

name = 'OTI'
column = 'OTI'
values = []
for i, group_mask in enumerate(group_masks.keys()):
    mask = (group_masks[group_mask]) & pd.notnull(Data_complete[column])
    v = Data_complete.loc[mask, column].values
    value = np.sum(v==1)
    if len(v)<20 or value==0:
        value_str = '-'
    else:
        number = len(v)
        perc = value/number
        value_str = '%d/%d (%.0f%%)' % (value, number, 100*perc)
    values.append(value_str)
description_dict[name] = values

name = 'Death'
column = 'death'
values = []
for i, group_mask in enumerate(group_masks.keys()):
    mask = (group_masks[group_mask]) & pd.notnull(Data_complete[column])
    v = Data_complete.loc[mask, column].values
    value = np.sum(v==1)
    if len(v)<20 or value==0:
        value_str = '-'
    else:
        number = len(v)
        perc = value/number
        value_str = '%d/%d (%.0f%%)' % (value, number, 100*perc)
    values.append(value_str)
description_dict[name] = values

name = 'death+OTI'
column = 'OTI+death' 
values = []
for i, group_mask in enumerate(group_masks.keys()):
    mask = (group_masks[group_mask]) & pd.notnull(Data_complete[column])
    v = Data_complete.loc[mask, column].values
    value = np.sum(v==1)
    if len(v)<20 or value==0:
        value_str = '-'
    else:
        number = len(v)
        perc = value/number
        value_str = '%d/%d (%.0f%%)' % (value, number, 100*perc)
    values.append(value_str)
description_dict[name] = values


## Dataframe
column_names = ['Control',
                'Control (age$<$%d)' % age_thr, 
                'Control (age$\ge$%d)' % age_thr,
                'Covid',
                'Covid (age$<$%d)' % age_thr, 
                'Covid (age$\ge$%d)' % age_thr, 
                'Covid ($\mathrm{\Delta t_{ons}}\le$%d)' % donset_th, 
                'Covid ($\mathrm{\Delta t_{ons}}>$%d)' % donset_th, 
                'Covid (WHO$\le$2)',
                'Covid (WHO$\ge$3)']

description_df = pd.DataFrame(description_dict.values(), index=description_dict.keys(), columns=column_names)


## Create directory and export table
foldername = 'DescriptiveStatistics/'
try:
    os.mkdir(path_results+foldername)
except OSError:
    print ("Directory already exists")
else:
    print ("Successfully created the directory")

filename = 'SummaryTable_AgeMin#%d_AgeMax#%d_new.xlsx' % (lower_bound, upper_bound)
description_df.to_excel(path_results+foldername+filename, engine='xlsxwriter')

# Boxplots: features vs outcome

In [ ]:
strat_variable = 'WHO'

fontsize = 9
ylabelsize = 7
xlabelsize = 7 
tex = True
axes_lines_w = 0.5
lines_w = 0.5
widths = 0.3
showfliers = False
showcaps = False

magnification = 0.68
meanprops = {'color':'black', 'markerfacecolor':'black', 'markeredgecolor':'black', 'marker':'.', 'ms':1.7}

name_feat_vs_outcome = 'features' + 'VSoutcome'

y_group_dem = ['age', 'delta_onset']
y_variables = ['age', 'delta_onset']
colors = [color_demographics]*len(y_group_dem)
y_group_fc = ['WBC/uL', 'Granulo/uL', 'Mono/uL', 'Mono DR IF', 'Mono DR %', 'Lymph/uL', 'RTE/uL', 'RTE % CD4']
y_group_fc += ['NK/uL', 'B CD19/uL', 'T CD3/uL', 'T CD4/uL', '% T CD4 HLADR+', 'T CD8/uL', '% T CD8 HLADR+']
y_variables += y_group_fc
colors += [color_immunecells]*len(y_group_fc)
y_group_ck = ['IL10', 'IL6', 'IL8', 'IL1B', 'IL2R', 'IP10', 'IFNGC']
y_variables += y_group_ck
colors += [color_cytokines]*len(y_group_ck)
y_group_bm = ['LDH', 'proADM', 'CRP']
y_variables += y_group_bm
colors += [color_bloodtests1]*len(y_group_bm)
y_group_score = ['SOFA', 'qCSI', 'NEWS', 'CCI (charlson comorbidity index)', '4C']
y_groups = [y_group_dem, y_group_fc, y_group_ck, y_group_bm, y_group_score]
y_variables += y_group_score
colors += [color_scores]*len(y_group_score)
new_labels = CF.change_names(y_variables, units=False)
n_variables = len(y_variables)
positions = np.arange(n_variables)

In [ ]:
# Sort variables according to AUC score
path_import = path_results+foldername_univ
filename = 'performances_test.xlsx'
df_performances = pd.read_excel(path_import+filename, engine='openpyxl', header=[0, 1], index_col=0)
index = [col.replace('LR: ', '') for col in df_performances.index]
df_performances.index = index
Data_l = df_performances.loc[y_variables, :].copy()

# Sorting
sorting_score = 'roc_auc_score'
y = Data_l[(sorting_score, 'Mean')].values
idx_sorted = []
for set_col in y_groups:
    sub_idx = np.array([i for i in range(len(y_variables)) if y_variables[i] in set_col])
    min_sub_idx = min(sub_idx)
    sub_idx_sorted = np.argsort(y[sub_idx])
    idx_sorted.extend(min_sub_idx + sub_idx_sorted)
idx_sorted = np.array(idx_sorted)

y_variables = list(np.array(y_variables)[idx_sorted])
new_labels = list(np.array(new_labels)[idx_sorted])

In [ ]:
y_groups_sorted = []
new_labels_groups_sorted = []
pos = 0
for y_group in y_groups:
    y_group_sorted = y_variables[pos:pos+len(y_group)]
    y_groups_sorted.append(y_group_sorted)
    new_labels_groups_sorted.append(new_labels[pos:pos+len(y_group)])
    pos += len(y_group)

In [ ]:
output = 'OTI+death'
group_names = ['dem', 'fc', 'ck', 'bm', 'scores']

ss = StandardScaler()
pt = PowerTransformer()

fig_height = 1.2
idx_color = 0
for idx_group, y_group in enumerate(y_groups_sorted):
    print('\n'.join(y_group))
    labels_group = new_labels_groups_sorted[idx_group]
    n_variables_group = len(y_group)
    positions = np.arange(n_variables_group)
    ratio = 5./n_variables_group
    fig_width = 0.95 + fig_height/ratio
    if n_variables_group<7:
        fig_width = 0.9 + fig_height/ratio
        
    CF.SetPlotParams(height=fig_height, width=fig_width,
                            fontsize=fontsize, ylabelsize=ylabelsize,
                            xlabelsize=xlabelsize, tex=tex, axes_lines_w=axes_lines_w, 
                            lines_w=lines_w, legendmarker=True)

    Data = DataInpatients[y_group+[output]].copy()

    plt.axhline(0, lw=0.5, color=Parameters.light_grey)

    # Make plot
    for i, variable in enumerate(Data):
        if variable!=output:

            Data_xy = Data[[variable,output]].copy().dropna()
            y = Data_xy[output].values
            x = Data_xy[variable].values
            x_norm = pt.fit_transform(ss.fit_transform(x.reshape(-1, 1))).reshape(-1,)
            x_norm_0 = x_norm[y==0]
            x_norm_1 = x_norm[y==1]

            bp = plt.boxplot(x=x_norm_0, positions=[i-0.18],
                        widths=widths, showfliers=showfliers, showcaps=showcaps, 
                           showmeans=True, meanprops=meanprops,patch_artist=True)
            for element in ['whiskers', 'fliers', 'means', 'medians', 'caps']:
                plt.setp(bp[element], color='black', lw=0.5, alpha=1.)      
            for box in bp['boxes']:
                box.set(color=color_0, linewidth=0.5, alpha=1.)
                box.set(facecolor=color_0, alpha=0.65)

            bp = plt.boxplot(x=x_norm_1, positions=[i+0.18],
                             widths=widths, showfliers=showfliers, showcaps=showcaps, 
                             showmeans=True, meanprops=meanprops,patch_artist=True)
            for element in ['whiskers', 'fliers', 'means', 'medians', 'caps']:
                plt.setp(bp[element], color='black', lw=0.5, alpha=1.)      
            for box in bp['boxes']:
                box.set(color=color_1, linewidth=0.5, alpha=0.65)
                box.set(facecolor=color_1, alpha=0.65)

    #plt.xticks(positions, labels_group, rotation=90)
    plt.xticks(positions, positions)
    plt.yticks([-3, -1.5, 0, 1.5, 3])
    plt.ylim([-3.2, 3.2])
    plt.ylabel('z score')

    ax = plt.gca()
    for xtick, color in zip(ax.get_xticklabels(), colors[idx_color:idx_color+n_variables_group]):
        xtick.set_color(color)
    idx_color += n_variables_group
    group_name = group_names[idx_group]
    name_feat_vs_outcome = group_name + 'VSoutcome'
    saving_str = path_figures + name_feat_vs_outcome + '.pdf'
    plt.savefig(saving_str, bbox_inches='tight')# dpi=1000,

    plt.show()

### Sex hist

In [ ]:
variable = 'sex'
output = 'OTI+death'

fig_height = 1.25
n_bins = 2
positions = np.arange(n_bins)
ratio = 3.
fig_width = 1. + fig_height/ratio

CF.SetPlotParams(height=fig_height, width=fig_width,
                fontsize=fontsize, ylabelsize=ylabelsize,
                xlabelsize=xlabelsize, tex=tex, axes_lines_w=axes_lines_w, 
                lines_w=lines_w, legendmarker=True)

Data = DataInpatients.copy()
Data_xy = Data[[variable,output]].copy().dropna()
y = Data_xy[output].values
x = Data_xy[variable].values
frac_male_0 = sum((x==0) & (y==0))/sum(y==0)
ci0_low, ci0_up = proportion_confint(sum((x==0) & (y==0)), sum(y==0), alpha=0.05, method='binom_test')
frac_male_1 = sum((x==0) & (y==1))/sum(y==1)
ci1_low, ci1_up = proportion_confint(sum((x==0) & (y==1)), sum(y==1), alpha=0.05, method='binom_test')
perc_male_0 = round(frac_male_0*100)
yerr_perc_male_0 = np.array([(frac_male_0 - ci0_low)*100, (ci0_up - frac_male_0)*100]).reshape(-1,1)
perc_male_1 = round(frac_male_1*100)
yerr_perc_male_1 = np.array([(frac_male_1 - ci1_low)*100, (ci1_up - frac_male_1)*100]).reshape(-1,1)
print('%s=0: %.0f%%'%(output, perc_male_0))
print('%s=1: %.0f%%'%(output, perc_male_1))

fig, ax = plt.subplots()

lines_w = 0.5
capsize = 0
widths = 0.25
ax.bar(0, perc_male_0, yerr=yerr_perc_male_0, color=color_0, alpha=0.65, width=widths, lw=lines_w, capsize=capsize)
ax.bar(0.7, perc_male_1, yerr=yerr_perc_male_1, color=color_1, alpha=0.65, width=widths, lw=lines_w, capsize=capsize)
plt.yticks([30, 60, 90])
plt.ylim([30, 90])
plt.xticks([0, 0.7], [0, 1], )
plt.xlim([-0.5, 1.2])
plt.ylabel('\% male')
saving_str = path_figures + 'sexVSoutput' + '.pdf'
plt.savefig(saving_str, bbox_inches='tight')# dpi=1000,
plt.show()

# Boxplots: features vs WHO

In [ ]:
strat_variable = 'WHO'

fontsize = 9
ylabelsize = 7
xlabelsize = 7 
tex = True
axes_lines_w = 0.5
lines_w = 0.5
widths = 0.3
showfliers = False
showcaps = False

magnification = 0.68
meanprops = {'color':'black', 'markerfacecolor':'black', 'markeredgecolor':'black', 'marker':'.', 'ms':1.7}

positions = [0., 1.75*widths, 3.5*widths, 5.25*widths, 7*widths]

### Flow cytometry features

In [ ]:
color = color_immunecells
colors = [color_control] + [color]*4
alphas = [0.8, 0.5, 0.5, 0.5, 0.5]
x_labels = ['control', 'WHO $\\le$ 1', 'WHO $=$ 2', 'WHO $=$ 3', 'WHO $\\ge$ 4']
x_labels = ['C', '1', '2', '3', '4']

#### Set 1

In [ ]:
# features
name_feat_vs_WHO = 'flowcyt1' + 'VSwho'
y_variables = ['NK/uL', 'B CD19/uL', 'T CD3/uL', 'T CD4/uL', '% T CD4 HLADR+', 'T CD8/uL', '% T CD8 HLADR+']
new_labels = CF.change_names(y_variables, units=False)
n_variables = len(y_variables)

# data
Data = DataInpatients.copy()
Data_c = DataOutpatients.copy()

# group 'WHO' values
Data.loc[Data[strat_variable]==0, strat_variable] = 1
Data.loc[Data[strat_variable]==5, strat_variable] = 4

In [ ]:
ratio = len(y_variables)/2.1

CF.SetPlotParams(magnification=magnification, ratio=ratio, fontsize=fontsize, ylabelsize=ylabelsize,
                 xlabelsize=xlabelsize, tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w, legendmarker=True)

fig, ax =  plt.subplots(n_variables, 1, sharex=True)

for i, var in enumerate(y_variables):

    input_variable = var


    # data_to_plot
    data_to_boxplot = []
    mask = Data_c[strat_variable].values==0
    v = Data_c.loc[mask, input_variable].dropna().values
    data_to_boxplot.append(v)
    #
    mask = Data[strat_variable].values==1
    v = Data.loc[mask, input_variable].dropna().values
    data_to_boxplot.append(v)
    #
    mask = Data[strat_variable].values==2
    v = Data.loc[mask, input_variable].dropna().values
    data_to_boxplot.append(v)
    #
    mask = Data[strat_variable].values==3
    v = Data.loc[mask, input_variable].dropna().values
    data_to_boxplot.append(v)
    #
    mask = Data[strat_variable].values==4
    v = Data.loc[mask, input_variable].dropna().values
    data_to_boxplot.append(v)

    idx = 0
    bp = ax[i].boxplot(x=data_to_boxplot[idx], positions=[positions[idx]],
                widths=widths, showfliers=showfliers, showcaps=showcaps, 
                       showmeans=False, meanprops=meanprops,patch_artist=True)
    for element in ['whiskers', 'fliers', 'means', 'medians', 'caps']:
        plt.setp(bp[element], color='black', lw=0.5, alpha=1.)      
    for box in bp['boxes']:
        box.set(color=colors[idx], linewidth=0.7, alpha=1.)
        box.set(facecolor = colors[idx], alpha=alphas[idx])

    idx = 1
    bp = ax[i].boxplot(x=data_to_boxplot[idx], positions=[positions[idx]],
                widths=widths, showfliers=showfliers, showcaps=showcaps, 
                       showmeans=False, meanprops=meanprops,patch_artist=True)
    for element in ['whiskers', 'fliers', 'means', 'medians', 'caps']:
        plt.setp(bp[element], color='black', lw=0.5, alpha=1.)      
    for box in bp['boxes']:
        box.set(color=colors[idx], linewidth=0.7, alpha=1.)
        box.set(facecolor = colors[idx], alpha=alphas[idx])

    idx = 2
    bp = ax[i].boxplot(x=data_to_boxplot[idx], positions=[positions[idx]],
                widths=widths, showfliers=showfliers, showcaps=showcaps, 
                       showmeans=False, meanprops=meanprops,patch_artist=True)
    for element in ['whiskers', 'fliers', 'means', 'medians', 'caps']:
        plt.setp(bp[element], color='black', lw=0.5, alpha=1.)      
    for box in bp['boxes']:
        box.set(color=colors[idx], linewidth=0.7, alpha=1.)
        box.set(facecolor = colors[idx], alpha=alphas[idx])

    idx = 3
    bp = ax[i].boxplot(x=data_to_boxplot[idx], positions=[positions[idx]],
                widths=widths, showfliers=showfliers, showcaps=showcaps, 
                       showmeans=False, meanprops=meanprops,patch_artist=True)
    for element in ['whiskers', 'fliers', 'means', 'medians', 'caps']:
        plt.setp(bp[element], color='black', lw=0.5, alpha=1.)      
    for box in bp['boxes']:
        box.set(color=colors[idx], linewidth=0.7, alpha=1.)
        box.set(facecolor = colors[idx], alpha=alphas[idx])

    idx = 4
    bp = ax[i].boxplot(x=data_to_boxplot[idx], positions=[positions[idx]],
                widths=widths, showfliers=showfliers, showcaps=showcaps, 
                       showmeans=False, meanprops=meanprops,patch_artist=True)
    for element in ['whiskers', 'fliers', 'means', 'medians', 'caps']:
        plt.setp(bp[element], color='black', lw=0.5, alpha=1.)      
    for box in bp['boxes']:
        box.set(color=colors[idx], linewidth=0.7, alpha=1.)
        box.set(facecolor = colors[idx], alpha=alphas[idx])

        
    my_string = ''
    for k in range(5):
        for j in range(k+1, 5):
            a = data_to_boxplot[k]
            b = data_to_boxplot[j]
            st_val, p_val = st.mannwhitneyu(a, b)
            perc_change = 100*(np.median(b)-np.median(a))/np.median(a)

            if k==0 and j==1: 
                # p values
                if p_val<0.05:
                    p_val_not = '$^{*\,}$'                
                    if p_val<0.01:
                        p_val_not = '$^{**\,}$'
                    if p_val<0.001:
                        p_val_not = '$^{***}$'
                    if perc_change<0:
                        s = 'C-1: -'
                    else:
                        s = 'C-1: +'
                    s = s + '$%.0f$' % (abs(perc_change))
                    s = s + '\%' + p_val_not
                else:
                    s = 'C-1: --'
                my_string = my_string+s+'\n'

            if k==0 and j==4: 
                # p values
                if p_val<0.05:
                    p_val_not = '$^{*\,}$'                
                    if p_val<0.01:
                        p_val_not = '$^{**\,}$'
                    if p_val<0.001:
                        p_val_not = '$^{***}$'
                    if perc_change<0:
                        s = 'C-4: -'
                    else:
                        s = 'C-4: +'
                    s = s + '$%.0f$' % (abs(perc_change))
                    s = s + '\%' + p_val_not
                else:
                    s = 'C-4: --'                
                my_string = my_string+s+'\n'

            if k==1 and j==2: 
                # p values
                if p_val<0.05:
                    p_val_not = '$^{*\,}$'                
                    if p_val<0.01:
                        p_val_not = '$^{**\,}$'
                    if p_val<0.001:
                        p_val_not = '$^{***}$'
                    if perc_change<0:
                        s = '1-2: -'
                    else:
                        s = '1-2: +'
                    s = s + '$%.0f$' % (abs(perc_change))
                    s = s + '\%' + p_val_not
                else:
                    s = '1-2: --'                

            if k==1 and j==4: 
                # p values
                if p_val<0.05:
                    p_val_not = '$^{*\,}$'                
                    if p_val<0.01:
                        p_val_not = '$^{**\,}$'
                    if p_val<0.001:
                        p_val_not = '$^{***}$'
                    if perc_change<0:
                        s = '1-4: -'
                    else:
                        s = '1-4: +'
                    s = s + '$%.0f$' % (abs(perc_change))
                    s = s + '\%' + p_val_not
                else:
                    s = '1-4: --'                
                my_string = my_string+s+'\n'

            if k==3 and j==4: 
                # p values
                if p_val<0.05:
                    p_val_not = '$^{*\,}$'                
                    if p_val<0.01:
                        p_val_not = '$^{**\,}$'
                    if p_val<0.001:
                        p_val_not = '$^{***}$'
                    if perc_change<0:
                        s = '3-4: -'
                    else:
                        s = '3-4: +'
                    s = s + '$%.0f$' % (abs(perc_change))
                    s = s + '\%' + p_val_not
                else:
                    s = '3-4: --'                

    ax[i].text(1.05, 0.5, my_string,
               horizontalalignment='left',
               verticalalignment='center',
               rotation='horizontal',
               transform=ax[i].transAxes,
               fontsize=ylabelsize)


    yScalarFormatter = cc.ScalarFormatterClass(m=1)
    yScalarFormatter.set_powerlimits((0,0))
    ax[i].yaxis.set_major_locator(plt.MaxNLocator(prune='both', nbins=1))
    ax[i].yaxis.set_major_formatter(yScalarFormatter)
    x_ticks_pos = np.array(positions)
    ax[i].set_xticks(x_ticks_pos)
    ax[i].set_xticklabels(x_labels)
    ax[i].set_xlabel(None)
    ax[i].tick_params(axis='y', pad=1.5)
    ax[i].yaxis.labelpad = 1.5
    new_ylabel = CF.change_names([input_variable], units=False)
    ax[i].set_ylabel(new_labels[i])
    ax[i].grid(False)

plt.xticks(x_ticks_pos, x_labels)
saving_str = path_figures + name_feat_vs_WHO + '.pdf'
plt.savefig(saving_str, bbox_inches='tight')# dpi=1000,

plt.show()

#### Set 2

In [ ]:
# features
name_feat_vs_WHO = 'flowcyt2' + 'VSwho'
y_variables = ['WBC/uL', 'Mono/uL', 'Mono DR IF', 'Mono DR %', 'Lymph/uL', 'RTE/uL', 'RTE % CD4']
new_labels = CF.change_names(y_variables, units=False)
n_variables = len(y_variables)

# data
Data = DataInpatients.copy()
Data_c = DataOutpatients.copy()

# group 'WHO' values
Data.loc[Data[strat_variable]==0, strat_variable] = 1
Data.loc[Data[strat_variable]==5, strat_variable] = 4

In [ ]:
ratio = len(y_variables)/2.1

CF.SetPlotParams(magnification=magnification, ratio=ratio, fontsize=fontsize, ylabelsize=ylabelsize,
                 xlabelsize=xlabelsize, tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w, legendmarker=True)

fig, ax =  plt.subplots(n_variables, 1, sharex=True)

for i, var in enumerate(y_variables):

    input_variable = var


    # data_to_plot
    data_to_boxplot = []
    mask = Data_c[strat_variable].values==0
    v = Data_c.loc[mask, input_variable].dropna().values
    data_to_boxplot.append(v)
    #
    mask = Data_complete[strat_variable].values>0
    v = Data_complete.loc[mask, input_variable].dropna().values
    #data_to_boxplot.append(v)
    #
    mask = Data[strat_variable].values==1
    v = Data.loc[mask, input_variable].dropna().values
    data_to_boxplot.append(v)
    #
    mask = Data[strat_variable].values==2
    v = Data.loc[mask, input_variable].dropna().values
    data_to_boxplot.append(v)
    #
    mask = Data[strat_variable].values==3
    v = Data.loc[mask, input_variable].dropna().values
    data_to_boxplot.append(v)
    #
    mask = Data[strat_variable].values==4
    v = Data.loc[mask, input_variable].dropna().values
    data_to_boxplot.append(v)

    idx = 0
    bp = ax[i].boxplot(x=data_to_boxplot[idx], positions=[positions[idx]],
                widths=widths, showfliers=showfliers, showcaps=showcaps, 
                       showmeans=False, meanprops=meanprops,patch_artist=True)
    for element in ['whiskers', 'fliers', 'means', 'medians', 'caps']:
        plt.setp(bp[element], color='black', lw=0.5, alpha=1.)      
    for box in bp['boxes']:
        box.set(color=colors[idx], linewidth=0.7, alpha=1.)
        box.set(facecolor = colors[idx], alpha=alphas[idx])

    idx = 1
    bp = ax[i].boxplot(x=data_to_boxplot[idx], positions=[positions[idx]],
                widths=widths, showfliers=showfliers, showcaps=showcaps, 
                       showmeans=False, meanprops=meanprops,patch_artist=True)
    for element in ['whiskers', 'fliers', 'means', 'medians', 'caps']:
        plt.setp(bp[element], color='black', lw=0.5, alpha=1.)      
    for box in bp['boxes']:
        box.set(color=colors[idx], linewidth=0.7, alpha=1.)
        box.set(facecolor = colors[idx], alpha=alphas[idx])

    idx = 2
    bp = ax[i].boxplot(x=data_to_boxplot[idx], positions=[positions[idx]],
                widths=widths, showfliers=showfliers, showcaps=showcaps, 
                       showmeans=False, meanprops=meanprops,patch_artist=True)
    for element in ['whiskers', 'fliers', 'means', 'medians', 'caps']:
        plt.setp(bp[element], color='black', lw=0.5, alpha=1.)      
    for box in bp['boxes']:
        box.set(color=colors[idx], linewidth=0.7, alpha=1.)
        box.set(facecolor = colors[idx], alpha=alphas[idx])

    idx = 3
    bp = ax[i].boxplot(x=data_to_boxplot[idx], positions=[positions[idx]],
                widths=widths, showfliers=showfliers, showcaps=showcaps, 
                       showmeans=False, meanprops=meanprops,patch_artist=True)
    for element in ['whiskers', 'fliers', 'means', 'medians', 'caps']:
        plt.setp(bp[element], color='black', lw=0.5, alpha=1.)      
    for box in bp['boxes']:
        box.set(color=colors[idx], linewidth=0.7, alpha=1.)
        box.set(facecolor = colors[idx], alpha=alphas[idx])

    idx = 4
    bp = ax[i].boxplot(x=data_to_boxplot[idx], positions=[positions[idx]],
                widths=widths, showfliers=showfliers, showcaps=showcaps, 
                       showmeans=False, meanprops=meanprops,patch_artist=True)
    for element in ['whiskers', 'fliers', 'means', 'medians', 'caps']:
        plt.setp(bp[element], color='black', lw=0.5, alpha=1.)      
    for box in bp['boxes']:
        box.set(color=colors[idx], linewidth=0.7, alpha=1.)
        box.set(facecolor = colors[idx], alpha=alphas[idx])

    my_string = ''
    for k in range(5):
        for j in range(k+1, 5):
            a = data_to_boxplot[k]
            b = data_to_boxplot[j]
            st_val, p_val = st.mannwhitneyu(a, b) 
            perc_change = 100*(np.median(b)-np.median(a))/np.median(a)

            if k==0 and j==1: 
                # p values
                if p_val<0.05:
                    p_val_not = '$^{*\,}$'                
                    if p_val<0.01:
                        p_val_not = '$^{**\,}$'
                    if p_val<0.001:
                        p_val_not = '$^{***}$'
                    if perc_change<0:
                        s = 'C-1: -'
                    else:
                        s = 'C-1: +'
                    s = s + '$%.0f$' % (abs(perc_change))
                    s = s + '\%' + p_val_not
                else:
                    s = 'C-1: --'
                my_string = my_string+s+'\n'

            if k==0 and j==4: 
                # p values
                if p_val<0.05:
                    p_val_not = '$^{*\,}$'                
                    if p_val<0.01:
                        p_val_not = '$^{**\,}$'
                    if p_val<0.001:
                        p_val_not = '$^{***}$'
                    if perc_change<0:
                        s = 'C-4: -'
                    else:
                        s = 'C-4: +'
                    s = s + '$%.0f$' % (abs(perc_change))
                    s = s + '\%' + p_val_not
                else:
                    s = 'C-4: --'                
                my_string = my_string+s+'\n'

            if k==1 and j==2: 
                # p values
                if p_val<0.05:
                    p_val_not = '$^{*\,}$'                
                    if p_val<0.01:
                        p_val_not = '$^{**\,}$'
                    if p_val<0.001:
                        p_val_not = '$^{***}$'
                    if perc_change<0:
                        s = '1-2: -'
                    else:
                        s = '1-2: +'
                    s = s + '$%.0f$' % (abs(perc_change))
                    s = s + '\%' + p_val_not
                else:
                    s = '1-2: --'                

            if k==1 and j==4: 
                # p values
                if p_val<0.05:
                    p_val_not = '$^{*\,}$'                
                    if p_val<0.01:
                        p_val_not = '$^{**\,}$'
                    if p_val<0.001:
                        p_val_not = '$^{***}$'
                    if perc_change<0:
                        s = '1-4: -'
                    else:
                        s = '1-4: +'
                    s = s + '$%.0f$' % (abs(perc_change))
                    s = s + '\%' + p_val_not
                else:
                    s = '1-4: --'                
                my_string = my_string+s+'\n'

            if k==3 and j==4: 
                # p values
                if p_val<0.05:
                    p_val_not = '$^{*\,}$'                
                    if p_val<0.01:
                        p_val_not = '$^{**\,}$'
                    if p_val<0.001:
                        p_val_not = '$^{***}$'
                    if perc_change<0:
                        s = '3-4: -'
                    else:
                        s = '3-4: +'
                    s = s + '$%.0f$' % (abs(perc_change))
                    s = s + '\%' + p_val_not
                else:
                    s = '3-4: --'                

    ax[i].text(1.05, 0.5, my_string,
               horizontalalignment='left',
               verticalalignment='center',
               rotation='horizontal',
               transform=ax[i].transAxes,
               fontsize=ylabelsize)


    yScalarFormatter = cc.ScalarFormatterClass(m=1)
    yScalarFormatter.set_powerlimits((0,0))
    ax[i].yaxis.set_major_locator(plt.MaxNLocator(prune='both', nbins=1))
    ax[i].yaxis.set_major_formatter(yScalarFormatter)
    x_ticks_pos = np.array(positions)
    ax[i].set_xticks(x_ticks_pos)
    ax[i].set_xticklabels(x_labels)
    ax[i].set_xlabel(None)
    ax[i].tick_params(axis='y', pad=1.5)
    ax[i].yaxis.labelpad = 1.5
    new_ylabel = CF.change_names([input_variable], units=False)
    ax[i].set_ylabel(new_labels[i])
    ax[i].grid(False)

plt.xticks(x_ticks_pos, x_labels)
saving_str = path_figures + name_feat_vs_WHO + '.pdf'
plt.savefig(saving_str, bbox_inches='tight')# dpi=1000,

plt.show()

### Cytokines

In [ ]:
color = color_cytokines
colors = [color]*4
alphas = [0.5, 0.5, 0.5, 0.5]
x_labels = ['1', '2', '3', '4']

In [ ]:
# features
name_feat_vs_WHO = 'cytokines' + 'VSwho'
y_variables = ['IL10', 'IL6', 'IL8', 'IL1B', 'IL2R', 'IP10', 'IFNGC']
new_labels = CF.change_names(y_variables, units=False)
n_variables = len(y_variables)

# data
Data = DataInpatients.copy()

# group 'WHO' values
Data.loc[Data[strat_variable]==0, strat_variable] = 1
Data.loc[Data[strat_variable]==5, strat_variable] = 4

In [ ]:
ratio = len(y_variables)/2.1

CF.SetPlotParams(magnification=magnification, ratio=ratio, fontsize=fontsize, ylabelsize=ylabelsize,
                 xlabelsize=xlabelsize, tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w, legendmarker=True)

fig, ax =  plt.subplots(n_variables, 1, sharex=True)

for i, var in enumerate(y_variables):

    input_variable = var


    # data_to_plot
    data_to_boxplot = []
    mask = Data[strat_variable].values>0
    v = Data.loc[mask, input_variable].dropna().values
    #data_to_boxplot.append(v)
    #
    mask = Data[strat_variable].values==1
    v = Data.loc[mask, input_variable].dropna().values
    data_to_boxplot.append(v)
    #
    mask = Data[strat_variable].values==2
    v = Data.loc[mask, input_variable].dropna().values
    data_to_boxplot.append(v)
    #
    mask = Data[strat_variable].values==3
    v = Data.loc[mask, input_variable].dropna().values
    data_to_boxplot.append(v)
    #
    mask = Data[strat_variable].values==4
    v = Data.loc[mask, input_variable].dropna().values
    data_to_boxplot.append(v)

    idx = 0
    bp = ax[i].boxplot(x=data_to_boxplot[idx], positions=[positions[idx]],
                widths=widths, showfliers=showfliers, showcaps=showcaps, 
                       showmeans=False, meanprops=meanprops,patch_artist=True)
    for element in ['whiskers', 'fliers', 'means', 'medians', 'caps']:
        plt.setp(bp[element], color='black', lw=0.5, alpha=1.)      
    for box in bp['boxes']:
        box.set(color=colors[idx], linewidth=0.7, alpha=1.)
        box.set(facecolor = colors[idx], alpha=alphas[idx])

    idx = 1
    bp = ax[i].boxplot(x=data_to_boxplot[idx], positions=[positions[idx]],
                widths=widths, showfliers=showfliers, showcaps=showcaps, 
                       showmeans=False, meanprops=meanprops,patch_artist=True)
    for element in ['whiskers', 'fliers', 'means', 'medians', 'caps']:
        plt.setp(bp[element], color='black', lw=0.5, alpha=1.)      
    for box in bp['boxes']:
        box.set(color=colors[idx], linewidth=0.7, alpha=1.)
        box.set(facecolor = colors[idx], alpha=alphas[idx])

    idx = 2
    bp = ax[i].boxplot(x=data_to_boxplot[idx], positions=[positions[idx]],
                widths=widths, showfliers=showfliers, showcaps=showcaps, 
                       showmeans=False, meanprops=meanprops,patch_artist=True)
    for element in ['whiskers', 'fliers', 'means', 'medians', 'caps']:
        plt.setp(bp[element], color='black', lw=0.5, alpha=1.)      
    for box in bp['boxes']:
        box.set(color=colors[idx], linewidth=0.7, alpha=1.)
        box.set(facecolor = colors[idx], alpha=alphas[idx])

    idx = 3
    bp = ax[i].boxplot(x=data_to_boxplot[idx], positions=[positions[idx]],
                widths=widths, showfliers=showfliers, showcaps=showcaps, 
                       showmeans=False, meanprops=meanprops,patch_artist=True)
    for element in ['whiskers', 'fliers', 'means', 'medians', 'caps']:
        plt.setp(bp[element], color='black', lw=0.5, alpha=1.)      
    for box in bp['boxes']:
        box.set(color=colors[idx], linewidth=0.7, alpha=1.)
        box.set(facecolor = colors[idx], alpha=alphas[idx])

    my_string = ''
    for k in range(4):
        for j in range(k+1, 4):
            a = data_to_boxplot[k]
            b = data_to_boxplot[j]
            st_val, p_val = st.mannwhitneyu(a, b)
            perc_change = 100*(np.median(b)-np.median(a))/np.median(a)

            if k==0 and j==1: 
                # p values
                if p_val<0.05:
                    p_val_not = '$^{*\,}$'                
                    if p_val<0.01:
                        p_val_not = '$^{**\,}$'
                    if p_val<0.001:
                        p_val_not = '$^{***}$'
                    if perc_change<0:
                        s = '1-2: -'
                    else:
                        s = '1-2: +'
                    s = s + '$%.0f$' % (abs(perc_change))
                    s = s + '\%' + p_val_not
                else:
                    s = '1-2: --'
                my_string = my_string+s+'\n'

            if k==0 and j==3: 
                # p values
                if p_val<0.05:
                    p_val_not = '$^{*\,}$'                
                    if p_val<0.01:
                        p_val_not = '$^{**\,}$'
                    if p_val<0.001:
                        p_val_not = '$^{***}$'
                    if perc_change<0:
                        s = '1-4: -'
                    else:
                        s = '1-4: +'
                    s = s + '$%.0f$' % (abs(perc_change))
                    s = s + '\%' + p_val_not
                else:
                    s = '1-4: --'                
                my_string = my_string+s+'\n'

            if k==2 and j==3: 
                # p values
                if p_val<0.05:
                    p_val_not = '$^{*\,}$'                
                    if p_val<0.01:
                        p_val_not = '$^{**\,}$'
                    if p_val<0.001:
                        p_val_not = '$^{***}$'
                    if perc_change<0:
                        s = '3-4: -'
                    else:
                        s = '3-4: +'
                    s = s + '$%.0f$' % (abs(perc_change))
                    s = s + '\%' + p_val_not
                else:
                    s = '3-4: --'                
                my_string = my_string+s+'\n'

    ax[i].text(1.05, 0.5, my_string,
               horizontalalignment='left',
               verticalalignment='center',
               rotation='horizontal',
               transform=ax[i].transAxes,
               fontsize=ylabelsize)


    yScalarFormatter = cc.ScalarFormatterClass(m=1)
    yScalarFormatter.set_powerlimits((0,0))
    ax[i].yaxis.set_major_locator(plt.MaxNLocator(prune='both', nbins=1))
    ax[i].yaxis.set_major_formatter(yScalarFormatter)
    x_ticks_pos = np.array(positions[:-1])
    ax[i].set_xticks(x_ticks_pos)
    ax[i].set_xticklabels(x_labels)
    ax[i].set_xlabel(None)
    ax[i].tick_params(axis='y', pad=1.5)
    ax[i].yaxis.labelpad = 1.5
    new_ylabel = CF.change_names([input_variable], units=False)
    ax[i].set_ylabel(new_labels[i])
    ax[i].grid(False)

plt.xticks(x_ticks_pos, x_labels)
saving_str = path_figures + name_feat_vs_WHO + '.pdf'
plt.savefig(saving_str, bbox_inches='tight')# dpi=1000,

plt.show()

### Biomarkers

In [ ]:
color = color_bloodtests2
colors = [color]*4
alphas = [0.5, 0.5, 0.5, 0.5]
x_labels = ['control', 'WHO $\\le$ 1', 'WHO $=$ 2', 'WHO $=$ 3', 'WHO $\\ge$ 4']
x_labels = ['1', '2', '3', '4']

In [ ]:
# features
name_feat_vs_WHO = 'biomarkers' + 'VSwho'
y_variables = ['LDH', 'proADM', 'CRP']
new_labels = CF.change_names(y_variables, units=False)
n_variables = len(y_variables)

# data
Data = DataInpatients.copy()

# group 'WHO' values
Data.loc[Data[strat_variable]==0, strat_variable] = 1
Data.loc[Data[strat_variable]==5, strat_variable] = 4

In [ ]:
ratio = len(y_variables)/2.

mag_corr = 1.01
CF.SetPlotParams(magnification=mag_corr*magnification, ratio=ratio, fontsize=fontsize, ylabelsize=ylabelsize,
                 xlabelsize=xlabelsize, tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w, legendmarker=True)

fig, ax =  plt.subplots(n_variables, 1, sharex=True)

for i, var in enumerate(y_variables):

    input_variable = var


    # data_to_plot
    data_to_boxplot = []
    mask = Data_complete[strat_variable].values>0
    v = Data_complete.loc[mask, input_variable].dropna().values
    #data_to_boxplot.append(v)
    #
    mask = Data[strat_variable].values==1
    v = Data.loc[mask, input_variable].dropna().values
    data_to_boxplot.append(v)
    #
    mask = Data[strat_variable].values==2
    v = Data.loc[mask, input_variable].dropna().values
    data_to_boxplot.append(v)
    #
    mask = Data[strat_variable].values==3
    v = Data.loc[mask, input_variable].dropna().values
    data_to_boxplot.append(v)
    #
    mask = Data[strat_variable].values==4
    v = Data.loc[mask, input_variable].dropna().values
    data_to_boxplot.append(v)

    idx = 0
    bp = ax[i].boxplot(x=data_to_boxplot[idx], positions=[positions[idx]],
                widths=widths, showfliers=showfliers, showcaps=showcaps, 
                       showmeans=False, meanprops=meanprops,patch_artist=True)
    for element in ['whiskers', 'fliers', 'means', 'medians', 'caps']:
        plt.setp(bp[element], color='black', lw=0.5, alpha=1.)      
    for box in bp['boxes']:
        box.set(color=colors[idx], linewidth=0.7, alpha=1.)
        box.set(facecolor = colors[idx], alpha=alphas[idx])

    idx = 1
    bp = ax[i].boxplot(x=data_to_boxplot[idx], positions=[positions[idx]],
                widths=widths, showfliers=showfliers, showcaps=showcaps, 
                       showmeans=False, meanprops=meanprops,patch_artist=True)
    for element in ['whiskers', 'fliers', 'means', 'medians', 'caps']:
        plt.setp(bp[element], color='black', lw=0.5, alpha=1.)      
    for box in bp['boxes']:
        box.set(color=colors[idx], linewidth=0.7, alpha=1.)
        box.set(facecolor = colors[idx], alpha=alphas[idx])

    idx = 2
    bp = ax[i].boxplot(x=data_to_boxplot[idx], positions=[positions[idx]],
                widths=widths, showfliers=showfliers, showcaps=showcaps, 
                       showmeans=False, meanprops=meanprops,patch_artist=True)
    for element in ['whiskers', 'fliers', 'means', 'medians', 'caps']:
        plt.setp(bp[element], color='black', lw=0.5, alpha=1.)      
    for box in bp['boxes']:
        box.set(color=colors[idx], linewidth=0.7, alpha=1.)
        box.set(facecolor = colors[idx], alpha=alphas[idx])

    idx = 3
    bp = ax[i].boxplot(x=data_to_boxplot[idx], positions=[positions[idx]],
                widths=widths, showfliers=showfliers, showcaps=showcaps, 
                       showmeans=False, meanprops=meanprops,patch_artist=True)
    for element in ['whiskers', 'fliers', 'means', 'medians', 'caps']:
        plt.setp(bp[element], color='black', lw=0.5, alpha=1.)      
    for box in bp['boxes']:
        box.set(color=colors[idx], linewidth=0.7, alpha=1.)
        box.set(facecolor = colors[idx], alpha=alphas[idx])


    my_string = ''
    for k in range(4):
        for j in range(k+1, 4):
            a = data_to_boxplot[k]
            b = data_to_boxplot[j]
            st_val, p_val = st.mannwhitneyu(a, b)
            perc_change = 100*(np.median(b)-np.median(a))/np.median(a)

            if k==0 and j==1: 
                # p values
                if p_val<0.05:
                    p_val_not = '$^{*\,}$'                
                    if p_val<0.01:
                        p_val_not = '$^{**\,}$'
                    if p_val<0.001:
                        p_val_not = '$^{***}$'
                    if perc_change<0:
                        s = '1-2: -'
                    else:
                        s = '1-2: +'
                    s = s + '$%.0f$' % (abs(perc_change))
                    s = s + '\%' + p_val_not
                else:
                    s = '1-2: --'
                my_string = my_string+s+'\n'

            if k==0 and j==3: 
                # p values
                if p_val<0.05:
                    p_val_not = '$^{*\,}$'                
                    if p_val<0.01:
                        p_val_not = '$^{**\,}$'
                    if p_val<0.001:
                        p_val_not = '$^{***}$'
                    if perc_change<0:
                        s = '1-4: -'
                    else:
                        s = '1-4: +'
                    s = s + '$%.0f$' % (abs(perc_change))
                    s = s + '\%' + p_val_not
                else:
                    s = '1-4: --'                
                my_string = my_string+s+'\n'

            if k==2 and j==3: 
                # p values
                if p_val<0.05:
                    p_val_not = '$^{*\,}$'                
                    if p_val<0.01:
                        p_val_not = '$^{**\,}$'
                    if p_val<0.001:
                        p_val_not = '$^{***}$'
                    if perc_change<0:
                        s = '3-4: -'
                    else:
                        s = '3-4: +'
                    s = s + '$%.0f$' % (abs(perc_change))
                    s = s + '\%' + p_val_not
                else:
                    s = '3-4: --'                
                my_string = my_string+s+'\n'

    ax[i].text(1.05, 0.5, my_string,
               horizontalalignment='left',
               verticalalignment='center',
               rotation='horizontal',
               transform=ax[i].transAxes,
               fontsize=ylabelsize)


    yScalarFormatter = cc.ScalarFormatterClass(m=1)
    yScalarFormatter.set_powerlimits((0,0))
    ax[i].yaxis.set_major_locator(plt.MaxNLocator(prune='both', nbins=1))
    ax[i].yaxis.set_major_formatter(yScalarFormatter)
    x_ticks_pos = np.array(positions[:-1])
    ax[i].set_xticks(x_ticks_pos)
    ax[i].set_xticklabels(x_labels)
    ax[i].set_xlabel(None)
    ax[i].tick_params(axis='y', pad=1.5)
    ax[i].yaxis.labelpad = 1.5
    new_ylabel = CF.change_names([input_variable], units=False)
    ax[i].set_ylabel(new_labels[i])
    ax[i].grid(False)

plt.xticks(x_ticks_pos, x_labels)
saving_str = path_figures + name_feat_vs_WHO + '.pdf'
plt.savefig(saving_str, bbox_inches='tight')# dpi=1000,

plt.show()

### Demographics

In [ ]:
color = color_0_light
colors = [color]*4
alphas = [0.5, 0.5, 0.5, 0.5]
x_labels = ['1', '2', '3', '4']

In [ ]:
# features
name_feat_vs_WHO = 'demographics' + 'VSwho'
y_variables = ['age', 'delta_onset']
new_labels = CF.change_names(y_variables, units=False)
n_variables = len(y_variables)

# data
Data = DataInpatients.copy()

# group 'WHO' values
Data.loc[Data[strat_variable]==0, strat_variable] = 1
Data.loc[Data[strat_variable]==5, strat_variable] = 4

In [ ]:
ratio = len(y_variables)/1.8

mag_corr = .95
CF.SetPlotParams(magnification=mag_corr*magnification, ratio=ratio, fontsize=fontsize, ylabelsize=ylabelsize,
                 xlabelsize=xlabelsize, tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w, legendmarker=True)

fig, ax =  plt.subplots(n_variables, 1, sharex=True)

for i, var in enumerate(y_variables):

    input_variable = var


    # data_to_plot
    data_to_boxplot = []
    mask = Data_complete[strat_variable].values>0
    v = Data_complete.loc[mask, input_variable].dropna().values
    #data_to_boxplot.append(v)
    #
    mask = Data[strat_variable].values==1
    v = Data.loc[mask, input_variable].dropna().values
    data_to_boxplot.append(v)
    #
    mask = Data[strat_variable].values==2
    v = Data.loc[mask, input_variable].dropna().values
    data_to_boxplot.append(v)
    #
    mask = Data[strat_variable].values==3
    v = Data.loc[mask, input_variable].dropna().values
    data_to_boxplot.append(v)
    #
    mask = Data[strat_variable].values==4
    v = Data.loc[mask, input_variable].dropna().values
    data_to_boxplot.append(v)

    idx = 0
    bp = ax[i].boxplot(x=data_to_boxplot[idx], positions=[positions[idx]],
                widths=widths, showfliers=showfliers, showcaps=showcaps, 
                       showmeans=False, meanprops=meanprops,patch_artist=True)
    for element in ['whiskers', 'fliers', 'means', 'medians', 'caps']:
        plt.setp(bp[element], color='black', lw=0.5, alpha=1.)      
    for box in bp['boxes']:
        box.set(color=colors[idx], linewidth=0.7, alpha=1.)
        box.set(facecolor = colors[idx], alpha=alphas[idx])

    idx = 1
    bp = ax[i].boxplot(x=data_to_boxplot[idx], positions=[positions[idx]],
                widths=widths, showfliers=showfliers, showcaps=showcaps, 
                       showmeans=False, meanprops=meanprops,patch_artist=True)
    for element in ['whiskers', 'fliers', 'means', 'medians', 'caps']:
        plt.setp(bp[element], color='black', lw=0.5, alpha=1.)      
    for box in bp['boxes']:
        box.set(color=colors[idx], linewidth=0.7, alpha=1.)
        box.set(facecolor = colors[idx], alpha=alphas[idx])

    idx = 2
    bp = ax[i].boxplot(x=data_to_boxplot[idx], positions=[positions[idx]],
                widths=widths, showfliers=showfliers, showcaps=showcaps, 
                       showmeans=False, meanprops=meanprops,patch_artist=True)
    for element in ['whiskers', 'fliers', 'means', 'medians', 'caps']:
        plt.setp(bp[element], color='black', lw=0.5, alpha=1.)      
    for box in bp['boxes']:
        box.set(color=colors[idx], linewidth=0.7, alpha=1.)
        box.set(facecolor = colors[idx], alpha=alphas[idx])

    idx = 3
    bp = ax[i].boxplot(x=data_to_boxplot[idx], positions=[positions[idx]],
                widths=widths, showfliers=showfliers, showcaps=showcaps, 
                       showmeans=False, meanprops=meanprops,patch_artist=True)
    for element in ['whiskers', 'fliers', 'means', 'medians', 'caps']:
        plt.setp(bp[element], color='black', lw=0.5, alpha=1.)      
    for box in bp['boxes']:
        box.set(color=colors[idx], linewidth=0.7, alpha=1.)
        box.set(facecolor = colors[idx], alpha=alphas[idx])


    my_string = ''
    for k in range(4):
        for j in range(k+1, 4):
            a = data_to_boxplot[k]
            b = data_to_boxplot[j]
            st_val, p_val = st.mannwhitneyu(a, b)
            perc_change = 100*(np.median(b)-np.median(a))/np.median(a)

            if k==0 and j==1: 
                # p values
                if p_val<0.05:
                    p_val_not = '$^{*\,}$'                
                    if p_val<0.01:
                        p_val_not = '$^{**\,}$'
                    if p_val<0.001:
                        p_val_not = '$^{***}$'
                    if perc_change<0:
                        s = '1-2: -'
                    else:
                        s = '1-2: +'
                    s = s + '$%.0f$' % (abs(perc_change))
                    s = s + '\%' + p_val_not
                else:
                    s = '1-2: --'
                my_string = my_string+s+'\n'

            if k==0 and j==3: 
                # p values
                if p_val<0.05:
                    p_val_not = '$^{*\,}$'                
                    if p_val<0.01:
                        p_val_not = '$^{**\,}$'
                    if p_val<0.001:
                        p_val_not = '$^{***}$'
                    if perc_change<0:
                        s = '1-4: -'
                    else:
                        s = '1-4: +'
                    s = s + '$%.0f$' % (abs(perc_change))
                    s = s + '\%' + p_val_not
                else:
                    s = '1-4: --'                
                my_string = my_string+s+'\n'

            if k==2 and j==3: 
                # p values
                if p_val<0.05:
                    p_val_not = '$^{*\,}$'                
                    if p_val<0.01:
                        p_val_not = '$^{**\,}$'
                    if p_val<0.001:
                        p_val_not = '$^{***}$'
                    if perc_change<0:
                        s = '3-4: -'
                    else:
                        s = '3-4: +'
                    s = s + '$%.0f$' % (abs(perc_change))
                    s = s + '\%' + p_val_not
                else:
                    s = '3-4: --'                
                my_string = my_string+s+'\n'

    ax[i].text(1.05, 0.5, my_string,
               horizontalalignment='left',
               verticalalignment='center',
               rotation='horizontal',
               transform=ax[i].transAxes,
               fontsize=ylabelsize)


    yScalarFormatter = cc.ScalarFormatterClass(m=1)
    yScalarFormatter.set_powerlimits((0,0))
    ax[i].yaxis.set_major_locator(plt.MaxNLocator(prune='both', nbins=1))
    ax[i].yaxis.set_major_formatter(yScalarFormatter)
    x_ticks_pos = np.array(positions[:-1])
    ax[i].set_xticks(x_ticks_pos)
    ax[i].set_xticklabels(x_labels)
    ax[i].set_xlabel(None)
    ax[i].tick_params(axis='y', pad=1.5)
    ax[i].yaxis.labelpad = 1.5
    new_ylabel = CF.change_names([input_variable], units=False)
    ax[i].set_ylabel(new_labels[i])
    ax[i].grid(False)

plt.xticks(x_ticks_pos, x_labels)
saving_str = path_figures + name_feat_vs_WHO + '.pdf'
plt.savefig(saving_str, bbox_inches='tight')# dpi=1000,

plt.show()

# Boxplots: in/outpatients flow cyt. features stratified by age

In [ ]:
strat_variable = 'age_strat'

fontsize = 9
ylabelsize = 7
xlabelsize = 7 
tex = True
axes_lines_w = 0.5
lines_w = 0.5
widths = 0.3
showfliers = False
showcaps = False
meanprops = {'color':'black', 'markerfacecolor':'black', 'markeredgecolor':'black', 'marker':'.', 'ms':1.7}

magnification = 0.68

positions = [0., 1.2*widths, 3.5*widths, 4.7*widths]

In [ ]:
color = color_immunecells
colors = [color_control, color, color_control, color]
alphas = [0.8, 0.5, 0.8, 0.5]
x_labels = ['L', 'H']
x_ticks_pos = [0.6*widths, 3.8*widths]

In [ ]:
age_th1 = 40
age_th2 = 60
age_th3 = 80

# data
Data = DataInpatients.copy()
Data_c = DataOutpatients.copy()

# Add age_strat column
Data['age_strat'] = np.nan
age = Data['age'].values
mask1 = (age>=age_th1) & (age<age_th2)
Data.loc[mask1,'age_strat'] = 1
mask2 = (age>=age_th2) & (age<age_th3)
Data.loc[mask2,'age_strat'] = 0
Data_c['age_strat'] = np.nan
age = Data_c['age'].values
mask1 = (age>=age_th1) & (age<age_th2)
Data_c.loc[mask1,'age_strat'] = 1
mask2 = (age>=age_th2) & (age<age_th3)
Data_c.loc[mask2,'age_strat'] = 0

#### Set 1

In [ ]:
name_feat_vs_popage = 'flowcyt1' + 'VSpopulation&age'
y_variables = ['NK/uL', 'B CD19/uL', 'T CD3/uL', 'T CD4/uL', '% T CD4 HLADR+', 'T CD8/uL', '% T CD8 HLADR+']
new_labels = CF.change_names(y_variables, units=False)
n_variables = len(y_variables)

In [ ]:
ratio = len(y_variables)/2.1

CF.SetPlotParams(magnification=magnification, ratio=ratio, fontsize=fontsize, ylabelsize=ylabelsize,
                 xlabelsize=xlabelsize, tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w, legendmarker=True)

fig, ax =  plt.subplots(n_variables, 1, sharex=True)

for i, var in enumerate(y_variables):

    input_variable = var


    # data_to_plot
    data_to_boxplot = []
    mask = (Data_c[strat_variable].values==1)
    v = Data_c.loc[mask, input_variable].dropna().values
    data_to_boxplot.append(v)
    #
    mask = (Data[strat_variable].values==1)
    v = Data.loc[mask, input_variable].dropna().values
    data_to_boxplot.append(v)
    #
    mask = (Data_c[strat_variable].values==0)
    v = Data_c.loc[mask, input_variable].dropna().values
    data_to_boxplot.append(v)
    #
    mask = (Data[strat_variable].values==0)
    v = Data.loc[mask, input_variable].dropna().values
    data_to_boxplot.append(v)

    for idx in range(len(data_to_boxplot)):
        bp = ax[i].boxplot(x=data_to_boxplot[idx], positions=[positions[idx]],
                    widths=widths, showfliers=showfliers, showcaps=showcaps, 
                        showmeans=False, meanprops=meanprops, patch_artist=True)
        for element in ['whiskers', 'fliers', 'means', 'medians', 'caps']:
            plt.setp(bp[element], color='black', lw=0.5, alpha=1.)   
        for box in bp['boxes']:
            box.set(color=colors[idx], linewidth=0.7, alpha=1.)
            box.set(facecolor = colors[idx], alpha=alphas[idx])

        
    my_string = ''
    for k in range(4):
        for j in range(k+1, 4):
            a = data_to_boxplot[k]
            b = data_to_boxplot[j]
            st_val, p_val = st.mannwhitneyu(a, b)
            perc_change = 100*(np.median(b)-np.median(a))/np.median(a)

            if k==0 and j==1: 
                # p values
                if p_val<0.05:
                    p_val_not = '$^{*\,}$'                
                    if p_val<0.01:
                        p_val_not = '$^{**\,}$'
                    if p_val<0.001:
                        p_val_not = '$^{***}$'
                    if perc_change<0:
                        s = 'L-L: -'
                    else:
                        s = 'L-L: +'
                    s = s + '$%.0f$' % (abs(perc_change))
                    s = s + '\%' + p_val_not
                else:
                    s = 'L-L: --'
                my_string = my_string+s+'\n'

            if k==2 and j==3: 
                # p values
                if p_val<0.05:
                    p_val_not = '$^{*\,}$'                
                    if p_val<0.01:
                        p_val_not = '$^{**\,}$'
                    if p_val<0.001:
                        p_val_not = '$^{***}$'
                    if perc_change<0:
                        s = 'H-H: -'
                    else:
                        s = 'H-H: +'
                    s = s + '$%.0f$' % (abs(perc_change))
                    s = s + '\%' + p_val_not
                else:
                    s = 'H-H: --'                
                my_string = my_string+s

    ax[i].text(1.05, 0.5, my_string,
               horizontalalignment='left',
               verticalalignment='center',
               rotation='horizontal',
               transform=ax[i].transAxes,
               fontsize=ylabelsize)


    yScalarFormatter = cc.ScalarFormatterClass(m=1)
    yScalarFormatter.set_powerlimits((0,0))
    ax[i].yaxis.set_major_locator(plt.MaxNLocator(prune='both', nbins=1))
    ax[i].yaxis.set_major_formatter(yScalarFormatter)
    ax[i].set_xticks(x_ticks_pos)
    ax[i].set_xticklabels(x_labels)
    ax[i].set_xlabel(None)
    ax[i].tick_params(axis='y', pad=1.5)
    ax[i].yaxis.labelpad = 1.5
    new_ylabel = CF.change_names([input_variable], units=False)
    ax[i].set_ylabel(new_labels[i])
    ax[i].grid(False)

plt.xticks(x_ticks_pos, x_labels)
saving_str = path_figures + name_feat_vs_popage + '.pdf'
plt.savefig(saving_str, bbox_inches='tight')# dpi=1000,
plt.show()

#### Set 2

In [ ]:
name_feat_vs_popage = 'flowcyt2' + 'VSpopulation&age'
y_variables = ['WBC/uL', 'Mono/uL', 'Mono DR IF', 'Mono DR %', 'Lymph/uL', 'RTE/uL', 'RTE % CD4']
new_labels = CF.change_names(y_variables, units=False)
n_variables = len(y_variables)

In [ ]:
ratio = len(y_variables)/2.1

CF.SetPlotParams(magnification=magnification, ratio=ratio, fontsize=fontsize, ylabelsize=ylabelsize,
                 xlabelsize=xlabelsize, tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w, legendmarker=True)

fig, ax =  plt.subplots(n_variables, 1, sharex=True)

for i, var in enumerate(y_variables):

    input_variable = var


    # data_to_plot
    data_to_boxplot = []
    mask = (Data_c[strat_variable].values==1)
    v = Data_c.loc[mask, input_variable].dropna().values
    data_to_boxplot.append(v)
    #
    mask = (Data[strat_variable].values==1)
    v = Data.loc[mask, input_variable].dropna().values
    data_to_boxplot.append(v)
    #
    mask = (Data_c[strat_variable].values==0)
    v = Data_c.loc[mask, input_variable].dropna().values
    data_to_boxplot.append(v)
    #
    mask = (Data[strat_variable].values==0)
    v = Data.loc[mask, input_variable].dropna().values
    data_to_boxplot.append(v)

    for idx in range(len(data_to_boxplot)):
        bp = ax[i].boxplot(x=data_to_boxplot[idx], positions=[positions[idx]],
                    widths=widths, showfliers=showfliers, showcaps=showcaps, 
                        showmeans=False, meanprops=meanprops, patch_artist=True)
        for element in ['whiskers', 'fliers', 'means', 'medians', 'caps']:
            plt.setp(bp[element], color='black', lw=0.5, alpha=1.)   
        for box in bp['boxes']:
            box.set(color=colors[idx], linewidth=0.7, alpha=1.)
            box.set(facecolor = colors[idx], alpha=alphas[idx])

        
    my_string = ''
    for k in range(4):
        for j in range(k+1, 4):
            a = data_to_boxplot[k]
            b = data_to_boxplot[j]
            st_val, p_val = st.mannwhitneyu(a, b) 
            perc_change = 100*(np.median(b)-np.median(a))/np.median(a)

            if k==0 and j==1: 
                # p values
                if p_val<0.05:
                    p_val_not = '$^{*\,}$'                
                    if p_val<0.01:
                        p_val_not = '$^{**\,}$'
                    if p_val<0.001:
                        p_val_not = '$^{***}$'
                    if perc_change<0:
                        s = 'L-L: -'
                    else:
                        s = 'L-L: +'
                    s = s + '$%.0f$' % (abs(perc_change))
                    s = s + '\%' + p_val_not
                else:
                    s = 'L-L: --'
                my_string = my_string+s+'\n'

            if k==2 and j==3: 
                # p values
                if p_val<0.05:
                    p_val_not = '$^{*\,}$'                
                    if p_val<0.01:
                        p_val_not = '$^{**\,}$'
                    if p_val<0.001:
                        p_val_not = '$^{***}$'
                    if perc_change<0:
                        s = 'H-H: -'
                    else:
                        s = 'H-H: +'
                    s = s + '$%.0f$' % (abs(perc_change))
                    s = s + '\%' + p_val_not
                else:
                    s = 'H-H: --'                
                my_string = my_string+s

    ax[i].text(1.05, 0.5, my_string,
               horizontalalignment='left',
               verticalalignment='center',
               rotation='horizontal',
               transform=ax[i].transAxes,
               fontsize=ylabelsize)

    yScalarFormatter = cc.ScalarFormatterClass(m=1)
    yScalarFormatter.set_powerlimits((0,0))
    ax[i].yaxis.set_major_locator(plt.MaxNLocator(prune='both', nbins=1))
    ax[i].yaxis.set_major_formatter(yScalarFormatter)
    ax[i].set_xticks(x_ticks_pos)
    ax[i].set_xticklabels(x_labels)
    ax[i].set_xlabel(None)
    ax[i].tick_params(axis='y', pad=1.5)
    ax[i].yaxis.labelpad = 1.5
    new_ylabel = CF.change_names([input_variable], units=False)
    ax[i].set_ylabel(new_labels[i])
    ax[i].grid(False)

plt.xticks(x_ticks_pos, x_labels)
saving_str = path_figures + name_feat_vs_popage + '.pdf'
plt.savefig(saving_str, bbox_inches='tight')# dpi=1000,

plt.show()

# Running quantiles vs age

In [ ]:
time_col = 'age'

fontsize = 9
ylabelsize = 7
xlabelsize = 7 
tex = True
axes_lines_w = 0.5
lines_w = 0.5
lw = 1.2

half_window = 15
min_age_val = 20
max_age_val = 110

max_time = max_age_val + half_window
x_min_display = 40
x_min_lim = x_min_display - 3
x_max_display = 100
x_max_lim = x_max_display + 3

x_min_display_c = 20
x_min_lim_c = x_min_display_c - 3
x_max_display_c = 80
x_max_lim_c = x_max_display_c + 3

color_ref = Parameters.green
alpha_ref = 0.5
alpha_shade_ref = 0.15
alpha_shade = 0.2

## Flow cyt. outpatients

#### Set 1

In [ ]:
# features
name_feat_vs_age = 'flowcyt1' + 'VSage_HalfWindow#%d' % half_window
y_variables = ['NK/uL', 'B CD19/uL', 'T CD3/uL', 'T CD4/uL', '% T CD4 HLADR+', 'T CD8/uL', '% T CD8 HLADR+',]
new_labels = CF.change_names(y_variables, units=False)
n_variables = len(y_variables)

# data
Data = DataInpatients.copy()
Data_c = DataOutpatients.copy()

In [ ]:
# Control only
magnification = 0.8
ratio = 5.5/2.
CF.SetPlotParams(magnification=magnification, ratio=ratio, fontsize=fontsize, ylabelsize=ylabelsize,
                 xlabelsize=xlabelsize, tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w, legendmarker=True)

fig, ax =  plt.subplots(n_variables, 1, sharex=True)

for i, var in enumerate(y_variables):
    x_c, y_q1, y_mode, y_q3 = CF.moving_quantiles(Data_c, var, time_col, half_window=half_window, min_time=x_min_lim_c, max_time=x_max_lim_c)
    time_mask = (x_c>=x_min_display_c) & (x_c<=x_max_display_c)
    x_c = x_c[time_mask]
    y_mode = y_mode[time_mask]
    y_q1 = y_q1[time_mask]
    y_q3 = y_q3[time_mask]
    y_min = min(y_q1)
    y_max = max(y_q3)
    y_range_w = abs(y_max-y_min)
    y_min = y_min - 0.25*y_range_w
    y_max = y_max + 0.25*y_range_w
    
    ax[i].plot(x_c, y_mode, '-', lw=lw, color=color_ref, alpha=alpha_ref)
    ax[i].fill_between(x_c, y_q1, y_q3, lw=lw, color=color_ref, alpha=alpha_shade_ref)
    ax[i].set_xticks([20, 40, 60, 80])
    ax[i].set_xlim([x_min_lim_c, x_max_lim_c])
    ax[i].grid(False)
    yScalarFormatter = cc.ScalarFormatterClass()
    yScalarFormatter.set_powerlimits((0,0))
    ax[i].yaxis.set_major_formatter(yScalarFormatter)
    ax[i].yaxis.set_major_locator(plt.MaxNLocator(prune='both', nbins=1))
    ax[i].set_ylabel(new_labels[i])
    ax[i].set_ylim([y_min, y_max])
    ax[i].set_ylabel(new_labels[i])
        
    # Compute difference and significance between ages
    timepoint_1 = 35
    Data_local = Data_c[[var, time_col]].copy().dropna()
    mask_1 = (Data_local[time_col].values>=timepoint_1-half_window) & (Data_local[time_col].values<=timepoint_1+half_window)
    samples_1 = Data_local.loc[mask_1, var].values
    mode_1 = np.quantile(samples_1, 0.5)
    timepoint_2 = 65
    mask_2 = (Data_local[time_col].values>=timepoint_2-half_window) & (Data_local[time_col].values<=timepoint_2+half_window)
    samples_2 = Data_local.loc[mask_2, var].values
    mode_2 = np.quantile(samples_2, 0.5)
    perc_change = 100*(mode_2-mode_1)/mode_1

    st_val, p_val = st.mannwhitneyu(samples_1, samples_2)
    
    if p_val<0.05:
        ax[i].plot([timepoint_1, timepoint_2], [mode_1, mode_2], ls='', marker='.', ms=4., color='black')
        p_val_not = '$^{*\,\,}$'
        if p_val<0.01:
            p_val_not = '$^{**\,}$'
        if p_val<0.001:
            p_val_not = '$^{***}$'
        if perc_change<0:
            s = '-'
        else:
            s = '+'
        s = s + '$%.0f$' % (abs(perc_change))
        s = s + '\%' + p_val_not
        ax[i].text(1.02, 0.5, s,
                   horizontalalignment='left',
                   verticalalignment='center',
                   rotation='horizontal',
                   transform=ax[i].transAxes,
                   fontsize=ylabelsize)

ax[i].set_xlabel('age')

saving_str = path_figures + name_feat_vs_age + '_outpatients' + '.pdf'
plt.savefig(saving_str, bbox_inches='tight')# dpi=1000,
plt.show()

#### Set 2

In [ ]:
# features
name_feat_vs_age = 'flowcyt2' + 'VSage_HalfWindow#%d' % half_window
y_variables = ['WBC/uL', 'Mono/uL', 'Mono DR IF', 'Mono DR %', 'Lymph/uL', 'RTE/uL', 'RTE % CD4']
new_labels = CF.change_names(y_variables, units=False)
n_variables = len(y_variables)

# data
Data = DataInpatients.copy()
Data_c = DataOutpatients.copy()

In [ ]:
# Control only
magnification = 0.8
ratio = 5.5/2.
CF.SetPlotParams(magnification=magnification, ratio=ratio, fontsize=fontsize, ylabelsize=ylabelsize,
                 xlabelsize=xlabelsize, tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w, legendmarker=True)

fig, ax =  plt.subplots(n_variables, 1, sharex=True)

for i, var in enumerate(y_variables):
    x_c, y_q1, y_mode, y_q3 = CF.moving_quantiles(Data_c, var, time_col, half_window=half_window, min_time=x_min_lim_c, max_time=x_max_lim_c)
    time_mask = (x_c>=x_min_display_c) & (x_c<=x_max_display_c)
    x_c = x_c[time_mask]
    y_mode = y_mode[time_mask]
    y_q1 = y_q1[time_mask]
    y_q3 = y_q3[time_mask]
    y_min = min(y_q1)
    y_max = max(y_q3)
    y_range_w = abs(y_max-y_min)
    y_min = y_min - 0.25*y_range_w
    y_max = y_max + 0.25*y_range_w
    
    ax[i].plot(x_c, y_mode, '-', lw=lw, color=color_ref, alpha=alpha_ref)
    ax[i].fill_between(x_c, y_q1, y_q3, lw=lw, color=color_ref, alpha=alpha_shade_ref) 
    ax[i].set_xticks([20, 40, 60, 80])
    ax[i].set_xlim([x_min_lim_c, x_max_lim_c])
    ax[i].grid(False)
    yScalarFormatter = cc.ScalarFormatterClass()
    yScalarFormatter.set_powerlimits((0,0))
    ax[i].yaxis.set_major_formatter(yScalarFormatter)
    ax[i].yaxis.set_major_locator(plt.MaxNLocator(prune='both', nbins=1))
    ax[i].set_ylabel(new_labels[i])
    ax[i].set_ylim([y_min, y_max])
    ax[i].set_ylabel(new_labels[i])
        
    # Compute difference and significance between ages
    timepoint_1 = 35
    Data_local = Data_c[[var, time_col]].copy().dropna()
    mask_1 = (Data_local[time_col].values>=timepoint_1-half_window) & (Data_local[time_col].values<=timepoint_1+half_window)
    samples_1 = Data_local.loc[mask_1, var].values
    mode_1 = np.quantile(samples_1, 0.5)
    timepoint_2 = 65
    mask_2 = (Data_local[time_col].values>=timepoint_2-half_window) & (Data_local[time_col].values<=timepoint_2+half_window)
    samples_2 = Data_local.loc[mask_2, var].values
    mode_2 = np.quantile(samples_2, 0.5)
    perc_change = 100*(mode_2-mode_1)/mode_1

    st_val, p_val = st.mannwhitneyu(samples_1, samples_2)
    
    if p_val<0.05:
        ax[i].plot([timepoint_1, timepoint_2], [mode_1, mode_2], ls='', marker='.', ms=4., color='black')
        p_val_not = '$^{*\,\,}$'
        if p_val<0.01:
            p_val_not = '$^{**\,}$'
        if p_val<0.001:
            p_val_not = '$^{***}$'
        if perc_change<0:
            s = '-'
        else:
            s = '+'
        s = s + '$%.0f$' % (abs(perc_change))
        s = s + '\%' + p_val_not
        ax[i].text(1.02, 0.5, s,
                   horizontalalignment='left',
                   verticalalignment='center',
                   rotation='horizontal',
                   transform=ax[i].transAxes,
                   fontsize=ylabelsize)

ax[i].set_xlabel('age')

saving_str = path_figures + name_feat_vs_age + '_outpatients' + '.pdf'
plt.savefig(saving_str, bbox_inches='tight')# dpi=1000,
plt.show()

## Flow cyt. inpatients

In [ ]:
color = color_immunecells

#### Set 1

In [ ]:
# features
name_feat_vs_age = 'flowcyt1' + 'VSage_HalfWindow#%d' % half_window
y_variables = ['NK/uL', 'B CD19/uL', 'T CD3/uL', 'T CD4/uL', '% T CD4 HLADR+', 'T CD8/uL', '% T CD8 HLADR+',]
new_labels = CF.change_names(y_variables, units=False)
n_variables = len(y_variables)

# data
Data = DataInpatients.copy()
Data_c = DataOutpatients.copy()

In [ ]:
# Control only
magnification = 0.75
ratio = 5.5/2.
CF.SetPlotParams(magnification=magnification, ratio=ratio, fontsize=fontsize, ylabelsize=ylabelsize,
                 xlabelsize=xlabelsize, tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w, legendmarker=True)

fig, ax =  plt.subplots(n_variables, 1, sharex=True)

for i, var in enumerate(y_variables):
    x, y_q1, y_mode, y_q3 = CF.moving_quantiles(Data, var, time_col, half_window=half_window, min_time=x_min_lim, max_time=x_max_lim)
    time_mask = (x>=x_min_display) & (x<=x_max_display)
    x = x[time_mask]
    y_mode = y_mode[time_mask]
    y_q1 = y_q1[time_mask]
    y_q3 = y_q3[time_mask]
    y_min = min(y_q1)
    y_max = max(y_q3)
    y_range_w = abs(y_max-y_min)
    y_min = y_min - 0.25*y_range_w
    y_max = y_max + 0.25*y_range_w
    
    ax[i].plot(x, y_mode, '-', lw=lw, color=color)
    ax[i].fill_between(x, y_q1, y_q3, lw=lw, color=color, alpha=alpha_shade)
    ax[i].set_xticks([40, 60, 80, 100])
    ax[i].set_xlim([x_min_lim, x_max_lim])
    ax[i].grid(False)
    yScalarFormatter = cc.ScalarFormatterClass()
    yScalarFormatter.set_powerlimits((0,0))
    ax[i].yaxis.set_major_formatter(yScalarFormatter)
    ax[i].yaxis.set_major_locator(plt.MaxNLocator(prune='both', nbins=1))
    ax[i].set_ylabel(new_labels[i])
    ax[i].set_ylim([y_min, y_max])
    ax[i].set_ylabel(new_labels[i])
        
    # Compute difference and significance between ages
    timepoint_1 = 55
    Data_local = Data[[var, time_col]].copy().dropna()
    mask_1 = (Data_local[time_col].values>=timepoint_1-half_window) & (Data_local[time_col].values<=timepoint_1+half_window)
    samples_1 = Data_local.loc[mask_1, var].values
    mode_1 = np.quantile(samples_1, 0.5)
    timepoint_2 = 85
    mask_2 = (Data_local[time_col].values>=timepoint_2-half_window) & (Data_local[time_col].values<=timepoint_2+half_window)
    samples_2 = Data_local.loc[mask_2, var].values
    mode_2 = np.quantile(samples_2, 0.5)
    perc_change = 100*(mode_2-mode_1)/mode_1

    st_val, p_val = st.mannwhitneyu(samples_1, samples_2)
    if p_val<0.05:
        ax[i].plot([timepoint_1, timepoint_2], [mode_1, mode_2], ls='', marker='.', ms=4., color='black')
        p_val_not = '$^{*\,\,}$'
        if p_val<0.01:
            p_val_not = '$^{**\,}$'
        if p_val<0.001:
            p_val_not = '$^{***}$'
        if perc_change<0:
            s = '-'
        else:
            s = '+'
        s = s + '$%.0f$' % (abs(perc_change))
        s = s + '\%' + p_val_not
        ax[i].text(1.02, 0.5, s,
                   horizontalalignment='left',
                   verticalalignment='center',
                   rotation='horizontal',
                   transform=ax[i].transAxes,
                   fontsize=ylabelsize)

ax[i].set_xlabel('age')

saving_str = path_figures + name_feat_vs_age + '.pdf'
plt.savefig(saving_str, bbox_inches='tight')# dpi=1000,
plt.show()

#### Set 2

In [ ]:
# features
name_feat_vs_age = 'flowcyt2' + 'VSage_HalfWindow#%d' % half_window
y_variables = ['WBC/uL', 'Mono/uL', 'Mono DR IF', 'Mono DR %', 'Lymph/uL', 'RTE/uL', 'RTE % CD4']
new_labels = CF.change_names(y_variables, units=False)
n_variables = len(y_variables)

# data
Data = DataInpatients.copy()
Data_c = DataOutpatients.copy()

In [ ]:
# Control only
magnification =  0.75
ratio = 5.5/2.
CF.SetPlotParams(magnification=magnification, ratio=ratio, fontsize=fontsize, ylabelsize=ylabelsize,
                 xlabelsize=xlabelsize, tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w, legendmarker=True)

fig, ax =  plt.subplots(n_variables, 1, sharex=True)

for i, var in enumerate(y_variables):
    x, y_q1, y_mode, y_q3 = CF.moving_quantiles(Data, var, time_col, half_window=half_window, min_time=x_min_lim, max_time=x_max_lim)
    time_mask = (x>=x_min_display) & (x<=x_max_display)
    x = x[time_mask]
    y_mode = y_mode[time_mask]
    y_q1 = y_q1[time_mask]
    y_q3 = y_q3[time_mask]
    y_min = min(y_q1)
    y_max = max(y_q3)
    y_range_w = abs(y_max-y_min)
    y_min = y_min - 0.25*y_range_w
    y_max = y_max + 0.25*y_range_w
    
    ax[i].plot(x, y_mode, '-', lw=lw, color=color)
    ax[i].fill_between(x, y_q1, y_q3, lw=lw, color=color, alpha=alpha_shade)
    ax[i].set_xticks([40, 60, 80, 100])
    ax[i].set_xlim([x_min_lim, x_max_lim])
    ax[i].grid(False)
    yScalarFormatter = cc.ScalarFormatterClass()
    yScalarFormatter.set_powerlimits((0,0))
    ax[i].yaxis.set_major_formatter(yScalarFormatter)
    ax[i].yaxis.set_major_locator(plt.MaxNLocator(prune='both', nbins=1))
    ax[i].set_ylabel(new_labels[i])
    ax[i].set_ylim([y_min, y_max])
    ax[i].set_ylabel(new_labels[i])
        
    # Compute difference and significance between ages
    timepoint_1 = 55
    Data_local = Data[[var, time_col]].copy().dropna()
    mask_1 = (Data_local[time_col].values>=timepoint_1-half_window) & (Data_local[time_col].values<=timepoint_1+half_window)
    samples_1 = Data_local.loc[mask_1, var].values
    mode_1 = np.quantile(samples_1, 0.5)
    timepoint_2 = 85
    mask_2 = (Data_local[time_col].values>=timepoint_2-half_window) & (Data_local[time_col].values<=timepoint_2+half_window)
    samples_2 = Data_local.loc[mask_2, var].values
    mode_2 = np.quantile(samples_2, 0.5)
    perc_change = 100*(mode_2-mode_1)/mode_1

    st_val, p_val = st.mannwhitneyu(samples_1, samples_2)
    
    if p_val<0.05:
        ax[i].plot([timepoint_1, timepoint_2], [mode_1, mode_2], ls='', marker='.', ms=4., color='black')
        p_val_not = '$^{*\,\,}$'
        if p_val<0.01:
            p_val_not = '$^{**\,}$'
        if p_val<0.001:
            p_val_not = '$^{***}$'
        if perc_change<0:
            s = '-'
        else:
            s = '+'
        s = s + '$%.0f$' % (abs(perc_change))
        s = s + '\%' + p_val_not
        ax[i].text(1.02, 0.5, s,
                   horizontalalignment='left',
                   verticalalignment='center',
                   rotation='horizontal',
                   transform=ax[i].transAxes,
                   fontsize=ylabelsize)

ax[i].set_xlabel('age')

saving_str = path_figures + name_feat_vs_age + '.pdf'
plt.savefig(saving_str, bbox_inches='tight')# dpi=1000,
plt.show()

## CK inpatients

In [ ]:
color = color_cytokines

ref_values = {'IL1B': 0.16, 
              'IL8': 16.2,
              'IL2R': 1435,
              'IL10': 3.8,
              'IP10': 222,
              'IFNGC': 0.99,
              'IL6': 6.4}

In [ ]:
# features
name_feat_vs_age = 'cytokines' + 'VSage_HalfWindow#%d' % half_window
y_variables = ['IL10', 'IL6', 'IL8', 'IL1B', 'IL2R', 'IP10', 'IFNGC']
new_labels = CF.change_names(y_variables, units=False)
n_variables = len(y_variables)

# data
Data = DataInpatients.copy()

In [ ]:
magnification =  0.75
ratio = 5.5/2.
CF.SetPlotParams(magnification=magnification, ratio=ratio, fontsize=fontsize, ylabelsize=ylabelsize,
                 xlabelsize=xlabelsize, tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w, legendmarker=True)

fig, ax =  plt.subplots(n_variables, 1, sharex=True)

for i, var in enumerate(y_variables):
    x, y_q1, y_mode, y_q3 = CF.moving_quantiles(Data, var, time_col, half_window=half_window, min_time=x_min_lim, max_time=x_max_lim)
    time_mask = (x>=x_min_display) & (x<=x_max_display)
    x = x[time_mask]
    y_mode = y_mode[time_mask]
    y_q1 = y_q1[time_mask]
    y_q3 = y_q3[time_mask]
    y_min = min(y_q1)
    y_max = max(y_q3)
    y_range_w = abs(y_max-y_min)
    y_min = y_min - 0.25*y_range_w
    y_max = y_max + 0.25*y_range_w
    
    ax[i].plot(x, y_mode, '-', lw=lw, color=color)
    ax[i].fill_between(x, y_q1, y_q3, lw=lw, color=color, alpha=alpha_shade)
    ax[i].set_xticks([40, 60, 80, 100])
    ax[i].set_xlim([x_min_lim, x_max_lim])
    ax[i].grid(False)
    yScalarFormatter = cc.ScalarFormatterClass()
    yScalarFormatter.set_powerlimits((0,0))
    ax[i].yaxis.set_major_formatter(yScalarFormatter)
    ax[i].yaxis.set_major_locator(plt.MaxNLocator(prune='both', nbins=1))
    ax[i].set_ylabel(new_labels[i])
    ax[i].set_ylim([y_min, y_max])
    ax[i].set_ylabel(new_labels[i])
    
    if var in ref_values.keys():
        y_ref = ref_values[var]
        exp = int(np.log10(y_max))
        factor = y_ref/(10.**exp)
        val = 0
        d = 0
        while val==0:
            d += 1
            s_0 = '%%.%df' % d
            s = s_0 % factor
            val = float(s)
        ref_label = '$<$%s (ref)' % (s)
        if i>0:
            ref_label = '$<$%s' % (s)
        ax[i].text(1.0, 1.03, ref_label,
            horizontalalignment='right',
            verticalalignment='bottom',
            rotation='horizontal',
            transform=ax[i].transAxes,
            fontsize=ylabelsize)
    
    # Compute difference and significance between ages
    timepoint_1 = 55
    Data_local = Data[[var, time_col]].copy().dropna()
    mask_1 = (Data_local[time_col].values>=timepoint_1-half_window) & (Data_local[time_col].values<=timepoint_1+half_window)
    samples_1 = Data_local.loc[mask_1, var].values
    mode_1 = np.quantile(samples_1, 0.5)
    timepoint_2 = 85
    mask_2 = (Data_local[time_col].values>=timepoint_2-half_window) & (Data_local[time_col].values<=timepoint_2+half_window)
    samples_2 = Data_local.loc[mask_2, var].values
    mode_2 = np.quantile(samples_2, 0.5)
    perc_change = 100*(mode_2-mode_1)/mode_1

    st_val, p_val = st.mannwhitneyu(samples_1, samples_2)
    
    if p_val<0.05:
        ax[i].plot([timepoint_1, timepoint_2], [mode_1, mode_2], ls='', marker='.', ms=4., color='black')
        p_val_not = '$^{*\,\,}$'
        if p_val<0.01:
            p_val_not = '$^{**\,}$'
        if p_val<0.001:
            p_val_not = '$^{***}$'
        if perc_change<0:
            s = '-'
        else:
            s = '+'
        s = s + '$%.0f$' % (abs(perc_change))
        s = s + '\%' + p_val_not
        ax[i].text(1.02, 0.5, s,
                   horizontalalignment='left',
                   verticalalignment='center',
                   rotation='horizontal',
                   transform=ax[i].transAxes,
                   fontsize=ylabelsize)

ax[i].set_xlabel('age')

saving_str = path_figures + name_feat_vs_age + '.pdf'
plt.savefig(saving_str, bbox_inches='tight')# dpi=1000,
plt.show()

## BM inpatients

In [ ]:
color = color_bloodtests2

ref_values = {'LDH': 480, 
              'proADM': 0.56,
              'CRP': 5}

In [ ]:
# features
name_feat_vs_age = 'biomarkers' + 'VSage_HalfWindow#%d' % half_window
y_variables = ['LDH', 'proADM', 'CRP']
new_labels = CF.change_names(y_variables, units=False)
n_variables = len(y_variables)

# data
Data = DataInpatients.copy()

In [ ]:
magnification = 0.77
ratio = 2.55/2.
CF.SetPlotParams(magnification=magnification, ratio=ratio, fontsize=fontsize, ylabelsize=ylabelsize,
                 xlabelsize=xlabelsize, tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w, legendmarker=True)

fig, ax =  plt.subplots(n_variables, 1, sharex=True)

for i, var in enumerate(y_variables):
    print(var)  
    x, y_q1, y_mode, y_q3 = CF.moving_quantiles(Data, var, time_col, half_window=half_window, min_time=x_min_lim, max_time=x_max_lim)
    time_mask = (x>=x_min_display) & (x<=x_max_display)
    x = x[time_mask]
    y_mode = y_mode[time_mask]
    y_q1 = y_q1[time_mask]
    y_q3 = y_q3[time_mask]
    y_min = min(y_q1)
    y_max = max(y_q3)
    y_range_w = abs(y_max-y_min)
    y_min = y_min - 0.25*y_range_w
    y_max = y_max + 0.25*y_range_w

    ax[i].plot(x, y_mode, '-', lw=lw, color=color)
    ax[i].fill_between(x, y_q1, y_q3, lw=lw, color=color, alpha=alpha_shade)
    ax[i].set_xticks([40, 60, 80, 100])
    ax[i].set_xlim([x_min_lim, x_max_lim])
    ax[i].grid(False)
    yScalarFormatter = cc.ScalarFormatterClass()
    yScalarFormatter.set_powerlimits((0,0))
    ax[i].yaxis.set_major_formatter(yScalarFormatter)
    ax[i].yaxis.set_major_locator(plt.MaxNLocator(prune='both', nbins=1))
    ax[i].set_ylabel(new_labels[i])
    ax[i].set_ylim([y_min, y_max])
    ax[i].set_ylabel(new_labels[i])

    if var in ref_values.keys():
        y_ref = ref_values[var]
        exp = int(np.log10(y_max))
        factor = y_ref/(10.**exp)
        val = 0
        d = 0
        while val==0:
            d += 1
            s_0 = '%%.%df' % d
            s = s_0 % factor
            val = float(s)
        ref_label = '$<$%s (ref)' % (s)
        if i>0:
            ref_label = '$<$%s' % (s)
        ax[i].text(1.0, 1.03, ref_label,
            horizontalalignment='right',
            verticalalignment='bottom',
            rotation='horizontal',
            transform=ax[i].transAxes,
            fontsize=ylabelsize)
    
    # Compute difference and significance between ages
    timepoint_1 = 55
    Data_local = Data[[var, time_col]].copy().dropna()
    mask_1 = (Data_local[time_col].values>=timepoint_1-half_window) & (Data_local[time_col].values<=timepoint_1+half_window)
    samples_1 = Data_local.loc[mask_1, var].values
    mode_1 = np.quantile(samples_1, 0.5)
    timepoint_2 = 85
    mask_2 = (Data_local[time_col].values>=timepoint_2-half_window) & (Data_local[time_col].values<=timepoint_2+half_window)
    samples_2 = Data_local.loc[mask_2, var].values
    mode_2 = np.quantile(samples_2, 0.5)
    perc_change = 100*(mode_2-mode_1)/mode_1

    st_val, p_val = st.mannwhitneyu(samples_1, samples_2)

    if p_val<0.05:
        ax[i].plot([timepoint_1, timepoint_2], [mode_1, mode_2], ls='', marker='.', ms=4., color='black')
        p_val_not = '$^{*\,\,}$'
        if p_val<0.01:
            p_val_not = '$^{**\,}$'
        if p_val<0.001:
            p_val_not = '$^{***}$'
        if perc_change<0:
            s = '-'
        else:
            s = '+'
        s = s + '$%.0f$' % (abs(perc_change))
        s = s + '\%' + p_val_not
        ax[i].text(1.02, 0.5, s,
                   horizontalalignment='left',
                   verticalalignment='center',
                   rotation='horizontal',
                   transform=ax[i].transAxes,
                   fontsize=ylabelsize)

ax[i].set_xlabel('age')

saving_str = path_figures + name_feat_vs_age + '.pdf'
plt.savefig(saving_str, bbox_inches='tight')# dpi=1000,
plt.show()

# Running quantiles vs delta onset

In [ ]:
time_col = 'delta_onset'

fontsize = 9
ylabelsize = 7
xlabelsize = 7 
tex = True
axes_lines_w = 0.5
lines_w = 0.5
lw = 1.2

half_window = 5
max_time = 20 + half_window
x_max_display = 20
x_max_lim = x_max_display + 1

color_ref = Parameters.green
alpha_ref = 0.5
alpha_shade_ref = 0.15
alpha_shade = 0.2

## Flow cyt. inpatients

In [ ]:
color = color_immunecells

#### Set 2

In [ ]:
# features
name_feat_vs_time = 'flowcyt1' + 'VSdonset_HalfWindow#%d' % half_window
y_variables = ['NK/uL', 'B CD19/uL', 'T CD3/uL', 'T CD4/uL', '% T CD4 HLADR+', 'T CD8/uL', '% T CD8 HLADR+',]
new_labels = CF.change_names(y_variables, units=False)
n_variables = len(y_variables)

# data
Data = DataInpatients.copy()
Data_c = DataOutpatients.copy()

In [ ]:
# Control only
magnification = 0.75
ratio = 5.5/2.
CF.SetPlotParams(magnification=magnification, ratio=ratio, fontsize=fontsize, ylabelsize=ylabelsize,
                 xlabelsize=xlabelsize, tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w, legendmarker=True)

fig, ax =  plt.subplots(n_variables, 1, sharex=True)

for i, var in enumerate(y_variables):
    x, y_q1, y_mode, y_q3 = CF.moving_quantiles(Data, var, time_col, half_window=half_window, max_time=x_max_lim)
    time_mask = (x>=0) & (x<=x_max_display)
    x = x[time_mask]
    y_mode = y_mode[time_mask]
    y_q1 = y_q1[time_mask]
    y_q3 = y_q3[time_mask]
    y_min = min(y_q1)
    y_max = max(y_q3)
    y_range_w = abs(y_max-y_min)
    y_min = y_min - 0.25*y_range_w
    y_max = y_max + 0.25*y_range_w
    
    ax[i].plot(x, y_mode, '-', lw=lw, color=color)
    ax[i].fill_between(x, y_q1, y_q3, lw=lw, color=color, alpha=alpha_shade)
    ax[i].set_xticks([0, 5, 10, 15, 20, 25])
    ax[i].set_xlim([0, x_max_lim])
    ax[i].grid(False)
    yScalarFormatter = cc.ScalarFormatterClass()
    yScalarFormatter.set_powerlimits((0,0))
    ax[i].yaxis.set_major_formatter(yScalarFormatter)
    ax[i].yaxis.set_major_locator(plt.MaxNLocator(prune='both', nbins=1))
    ax[i].set_ylabel(new_labels[i])
    ax[i].set_ylim([y_min, y_max])
    ax[i].set_ylabel(new_labels[i])
        
    # Compute difference and significance between ages
    timepoint_1 = 5
    Data_local = Data[[var, time_col]].copy().dropna()
    mask_1 = (Data_local[time_col].values>=timepoint_1-half_window) & (Data_local[time_col].values<=timepoint_1+half_window)
    samples_1 = Data_local.loc[mask_1, var].values
    mode_1 = np.quantile(samples_1, 0.5)
    timepoint_2 = 15
    mask_2 = (Data_local[time_col].values>=timepoint_2-half_window) & (Data_local[time_col].values<=timepoint_2+half_window)
    samples_2 = Data_local.loc[mask_2, var].values
    mode_2 = np.quantile(samples_2, 0.5)
    perc_change = 100*(mode_2-mode_1)/mode_1

    st_val, p_val = st.mannwhitneyu(samples_1, samples_2)

    if p_val<0.05:
        ax[i].plot([timepoint_1, timepoint_2], [mode_1, mode_2], ls='', marker='.', ms=4., color='black')
        p_val_not = '$^{*\,\,}$'
        if p_val<0.01:
            p_val_not = '$^{**\,}$'
        if p_val<0.001:
            p_val_not = '$^{***}$'
        if perc_change<0:
            s = '-'
        else:
            s = '+'
        s = s + '$%.0f$' % (abs(perc_change))
        s = s + '\%' + p_val_not
        ax[i].text(1.02, 0.5, s,
                   horizontalalignment='left',
                   verticalalignment='center',
                   rotation='horizontal',
                   transform=ax[i].transAxes,
                   fontsize=ylabelsize)

ax[i].set_xlabel('$\mathrm{\Delta t_{ons}}$')
saving_str = path_figures + name_feat_vs_time + '.pdf'
plt.savefig(saving_str, bbox_inches='tight')# dpi=1000,
plt.show()

#### Set 2

In [ ]:
# features
name_feat_vs_time = 'flowcyt2' + 'VSdonset_HalfWindow#%d' % half_window
y_variables = ['WBC/uL', 'Mono/uL', 'Mono DR IF', 'Mono DR %', 'Lymph/uL', 'RTE/uL', 'RTE % CD4']
new_labels = CF.change_names(y_variables, units=False)
n_variables = len(y_variables)

# data
Data = DataInpatients.copy()
Data_c = DataOutpatients.copy()

In [ ]:
magnification = 0.75
ratio = 5.5/2.
CF.SetPlotParams(magnification=magnification, ratio=ratio, fontsize=fontsize, ylabelsize=ylabelsize,
                 xlabelsize=xlabelsize, tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w, legendmarker=True)

fig, ax =  plt.subplots(n_variables, 1, sharex=True)

for i, var in enumerate(y_variables):
    x, y_q1, y_mode, y_q3 = CF.moving_quantiles(Data, var, time_col, half_window=half_window, max_time=x_max_lim)
    time_mask = (x>=0) & (x<=x_max_display)
    x = x[time_mask]
    y_mode = y_mode[time_mask]
    y_q1 = y_q1[time_mask]
    y_q3 = y_q3[time_mask]
    y_min = min(y_q1)
    y_max = max(y_q3)
    y_range_w = abs(y_max-y_min)
    y_min = y_min - 0.25*y_range_w
    y_max = y_max + 0.25*y_range_w
    
    ax[i].plot(x, y_mode, '-', lw=lw, color=color)
    ax[i].fill_between(x, y_q1, y_q3, lw=lw, color=color, alpha=alpha_shade)
    ax[i].set_xticks([0, 5, 10, 15, 20, 25])
    ax[i].set_xlim([0, x_max_lim])
    ax[i].grid(False)
    yScalarFormatter = cc.ScalarFormatterClass()
    yScalarFormatter.set_powerlimits((0,0))
    ax[i].yaxis.set_major_formatter(yScalarFormatter)
    ax[i].yaxis.set_major_locator(plt.MaxNLocator(prune='both', nbins=1))
    ax[i].set_ylabel(new_labels[i])
    ax[i].set_ylim([y_min, y_max])
    ax[i].set_ylabel(new_labels[i])
        
    # Compute difference and significance between ages
    timepoint_1 = 5
    Data_local = Data[[var, time_col]].copy().dropna()
    mask_1 = (Data_local[time_col].values>=timepoint_1-half_window) & (Data_local[time_col].values<=timepoint_1+half_window)
    samples_1 = Data_local.loc[mask_1, var].values
    mode_1 = np.quantile(samples_1, 0.5)
    timepoint_2 = 15
    mask_2 = (Data_local[time_col].values>=timepoint_2-half_window) & (Data_local[time_col].values<=timepoint_2+half_window)
    samples_2 = Data_local.loc[mask_2, var].values
    mode_2 = np.quantile(samples_2, 0.5)
    perc_change = 100*(mode_2-mode_1)/mode_1

    st_val, p_val = st.mannwhitneyu(samples_1, samples_2)
    if p_val<0.05:
        ax[i].plot([timepoint_1, timepoint_2], [mode_1, mode_2], ls='', marker='.', ms=4., color='black')
        p_val_not = '$^{*\,\,}$'
        if p_val<0.01:
            p_val_not = '$^{**\,}$'
        if p_val<0.001:
            p_val_not = '$^{***}$'
        if perc_change<0:
            s = '-'
        else:
            s = '+'
        s = s + '$%.0f$' % (abs(perc_change))
        s = s + '\%' + p_val_not
        ax[i].text(1.02, 0.5, s,
                   horizontalalignment='left',
                   verticalalignment='center',
                   rotation='horizontal',
                   transform=ax[i].transAxes,
                   fontsize=ylabelsize)

ax[i].set_xlabel('$\mathrm{\Delta t_{ons}}$')

saving_str = path_figures + name_feat_vs_time + '.pdf'
plt.savefig(saving_str, bbox_inches='tight')# dpi=1000,
plt.show()

## CK inpatients

In [ ]:
color = color_cytokines

ref_values = {'IL1B': 0.16, 
              'IL8': 16.2,
              'IL2R': 1435,
              'IL10': 3.8,
              'IP10': 222,
              'IFNGC': 0.99,
              'IL6': 6.4}

In [ ]:
# features
name_feat_vs_time = 'cytokines' + 'VSdonset_HalfWindow#%d' % half_window
y_variables = ['IL10', 'IL6', 'IL8', 'IL1B', 'IL2R', 'IP10', 'IFNGC']
new_labels = CF.change_names(y_variables, units=False)
n_variables = len(y_variables)

# data
Data = DataInpatients.copy()

In [ ]:
magnification = 0.75
ratio = 5.5/2.
CF.SetPlotParams(magnification=magnification, ratio=ratio, fontsize=fontsize, ylabelsize=ylabelsize,
                 xlabelsize=xlabelsize, tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w, legendmarker=True)

fig, ax =  plt.subplots(n_variables, 1, sharex=True)

for i, var in enumerate(y_variables):
    x, y_q1, y_mode, y_q3 = CF.moving_quantiles(Data, var, time_col, half_window=half_window, max_time=x_max_lim)
    time_mask = (x>=0) & (x<=x_max_display)
    x = x[time_mask]
    y_mode = y_mode[time_mask]
    y_q1 = y_q1[time_mask]
    y_q3 = y_q3[time_mask]
    y_min = min(y_q1)
    y_max = max(y_q3)
    y_range_w = abs(y_max-y_min)
    y_min = y_min - 0.25*y_range_w
    y_max = y_max + 0.25*y_range_w
    
    ax[i].plot(x, y_mode, '-', lw=lw, color=color)
    ax[i].fill_between(x, y_q1, y_q3, lw=lw, color=color, alpha=alpha_shade)
    ax[i].set_xticks([0, 5, 10, 15, 20, 25])
    ax[i].set_xlim([0, x_max_lim])
    ax[i].grid(False)
    yScalarFormatter = cc.ScalarFormatterClass()
    yScalarFormatter.set_powerlimits((0,0))
    ax[i].yaxis.set_major_formatter(yScalarFormatter)
    ax[i].yaxis.set_major_locator(plt.MaxNLocator(prune='both', nbins=1))
    ax[i].set_ylabel(new_labels[i])
    ax[i].set_ylim([y_min, y_max])
    ax[i].set_ylabel(new_labels[i])
        
    if var in ref_values.keys():
        y_ref = ref_values[var]
        exp = int(np.log10(y_max))
        factor = y_ref/(10.**exp)
        val = 0
        d = 0
        while val==0:
            d += 1
            s_0 = '%%.%df' % d
            s = s_0 % factor
            val = float(s)
        ref_label = '$<$%s (ref)' % (s)
        if i>0:
            ref_label = '$<$%s' % (s)
        ax[i].text(1.0, 1.03, ref_label,
            horizontalalignment='right',
            verticalalignment='bottom',
            rotation='horizontal',
            transform=ax[i].transAxes,
            fontsize=ylabelsize)
        
    # Compute difference and significance between ages
    timepoint_1 = 5
    Data_local = Data[[var, time_col]].copy().dropna()
    mask_1 = (Data_local[time_col].values>=timepoint_1-half_window) & (Data_local[time_col].values<=timepoint_1+half_window)
    samples_1 = Data_local.loc[mask_1, var].values
    mode_1 = np.quantile(samples_1, 0.5)
    timepoint_2 = 15
    mask_2 = (Data_local[time_col].values>=timepoint_2-half_window) & (Data_local[time_col].values<=timepoint_2+half_window)
    samples_2 = Data_local.loc[mask_2, var].values
    mode_2 = np.quantile(samples_2, 0.5)
    perc_change = 100*(mode_2-mode_1)/mode_1

    st_val, p_val = st.mannwhitneyu(samples_1, samples_2)

    if p_val<0.05:
        ax[i].plot([timepoint_1, timepoint_2], [mode_1, mode_2], ls='', marker='.', ms=4., color='black')
        p_val_not = '$^{*\,\,}$'
        if p_val<0.01:
            p_val_not = '$^{**\,}$'
        if p_val<0.001:
            p_val_not = '$^{***}$'
        if perc_change<0:
            s = '-'
        else:
            s = '+'
        s = s + '$%.0f$' % (abs(perc_change))
        s = s + '\%' + p_val_not
        ax[i].text(1.02, 0.5, s,
                   horizontalalignment='left',
                   verticalalignment='center',
                   rotation='horizontal',
                   transform=ax[i].transAxes,
                   fontsize=ylabelsize)

ax[i].set_xlabel('$\mathrm{\Delta t_{ons}}$')

saving_str = path_figures + name_feat_vs_time + '.pdf'
plt.savefig(saving_str, bbox_inches='tight')# dpi=1000,
plt.show()

## BM inpatients

In [ ]:
color = color_bloodtests2

ref_values = {'LDH': 480, 
              'proADM': 0.56,
              'CRP': 5}

In [ ]:
# features
name_feat_vs_time = 'biomarkers' + 'VSdonset_HalfWindow#%d' % half_window
y_variables = ['LDH', 'proADM', 'CRP']
new_labels = CF.change_names(y_variables, units=False)
n_variables = len(y_variables)

# data
Data = DataInpatients.copy()

In [ ]:
magnification = 0.77
ratio = 2.55/2.

CF.SetPlotParams(magnification=magnification, ratio=ratio, fontsize=fontsize, ylabelsize=ylabelsize,
                 xlabelsize=xlabelsize, tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w, legendmarker=True)

fig, ax =  plt.subplots(n_variables, 1, sharex=True)

for i, var in enumerate(y_variables):
    x, y_q1, y_mode, y_q3 = CF.moving_quantiles(Data, var, time_col, half_window=half_window, max_time=x_max_lim)
    time_mask = (x>=0) & (x<=x_max_display)
    x = x[time_mask]
    y_mode = y_mode[time_mask]
    y_q1 = y_q1[time_mask]
    y_q3 = y_q3[time_mask]
    y_min = min(y_q1)
    y_max = max(y_q3)
    y_range_w = abs(y_max-y_min)
    y_min = y_min - 0.25*y_range_w
    y_max = y_max + 0.25*y_range_w
    
    ax[i].plot(x, y_mode, '-', lw=lw, color=color)
    ax[i].fill_between(x, y_q1, y_q3, lw=lw, color=color, alpha=alpha_shade)
    ax[i].set_xticks([0, 5, 10, 15, 20, 25])
    ax[i].set_xlim([0, x_max_lim])
    ax[i].grid(False)
    yScalarFormatter = cc.ScalarFormatterClass()
    yScalarFormatter.set_powerlimits((0,0))
    ax[i].yaxis.set_major_formatter(yScalarFormatter)
    ax[i].yaxis.set_major_locator(plt.MaxNLocator(prune='both', nbins=1))
    ax[i].set_ylabel(new_labels[i])
    ax[i].set_ylim([y_min, y_max])
    ax[i].set_ylabel(new_labels[i])
        
    if var in ref_values.keys():
        y_ref = ref_values[var]
        exp = int(np.log10(y_max))
        factor = y_ref/(10.**exp)
        val = 0
        d = 0
        while val==0:
            d += 1
            s_0 = '%%.%df' % d
            s = s_0 % factor
            val = float(s)
        ref_label = '$<$%s (ref)' % (s)
        if i>0:
            ref_label = '$<$%s' % (s)
        ax[i].text(1.0, 1.03, ref_label,
            horizontalalignment='right',
            verticalalignment='bottom',
            rotation='horizontal',
            transform=ax[i].transAxes,
            fontsize=ylabelsize)
        
    # Compute difference and significance between ages
    timepoint_1 = 5
    Data_local = Data[[var, time_col]].copy().dropna()
    mask_1 = (Data_local[time_col].values>=timepoint_1-half_window) & (Data_local[time_col].values<=timepoint_1+half_window)
    samples_1 = Data_local.loc[mask_1, var].values
    mode_1 = np.quantile(samples_1, 0.5)
    timepoint_2 = 15
    mask_2 = (Data_local[time_col].values>=timepoint_2-half_window) & (Data_local[time_col].values<=timepoint_2+half_window)
    samples_2 = Data_local.loc[mask_2, var].values
    mode_2 = np.quantile(samples_2, 0.5)
    perc_change = 100*(mode_2-mode_1)/mode_1

    st_val, p_val = st.mannwhitneyu(samples_1, samples_2)
    
    if p_val<0.05:
        ax[i].plot([timepoint_1, timepoint_2], [mode_1, mode_2], ls='', marker='.', ms=4., color='black')
        p_val_not = '$^{*\,\,}$'
        if p_val<0.01:
            p_val_not = '$^{**\,}$'
        if p_val<0.001:
            p_val_not = '$^{***}$'
        if perc_change<0:
            s = '-'
        else:
            s = '+'
        s = s + '$%.0f$' % (abs(perc_change))
        s = s + '\%' + p_val_not
        ax[i].text(1.02, 0.5, s,
                   horizontalalignment='left',
                   verticalalignment='center',
                   rotation='horizontal',
                   transform=ax[i].transAxes,
                   fontsize=ylabelsize)

ax[i].set_xlabel('$\mathrm{\Delta t_{ons}}$')

saving_str = path_figures + name_feat_vs_time + '.pdf'
plt.savefig(saving_str, bbox_inches='tight')# dpi=1000,
plt.show()